# LIBRARY

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
# Untuk keperluan Database
import sqlite3
import uuid
from datetime import datetime

#Untuk keperluan model sistem rekomendasi
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Untuk kuisioner
import ipywidgets as widgets
from IPython.display import display

# Untuk Content-based Filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
# Tampilkan semua baris
pd.set_option('display.max_rows', None)
# Tampilkan semua kolom
pd.set_option('display.max_columns', None)
# Tampilkan isi sel sepanjang apapun
pd.set_option('display.max_colwidth', None)
# Jangan potong tampilan secara horizontal
pd.set_option('display.expand_frame_repr', True)
# Lebarkan tampilan ke batas maksimum
pd.set_option('display.width', 1000)

# VIEW DATABASE

In [ ]:
# View database
db_path = "/content/bone_recommender.db"

# Connect ke database
conn = sqlite3.connect(db_path)

# Baca tabel ke DataFrame
df = pd.read_sql_query("SELECT * FROM mst_tbl", conn)

# Tampilkan sebagai tabel
df

,user_id,created_at,kategori,umur,gender,domisili,INN-CE1,SE-M1,NACH-FF2,LOC-I1,...,CON-1,OPE-1,AGR-1,NEU-3,EXT-3,OPE-2,AGR-3,NEU-2,CON-3,EXT-1
0,89a5a8ff-b48a-405a-bd2a-efb9e1423526,2025-12-29 15:42:08.177049,Calon Wirausahawan,18,Laki-laki,Kabupaten Aceh Barat,5,5,4,4,...,3,4,5,4,3,4,5,4,3,5
1,ef491c40-e42d-47ca-92c6-33836869aaab,2025-12-29 16:22:26.476166,Calon Wirausahawan,18,Laki-laki,Kabupaten Aceh Barat,5,3,3,4,...,3,4,3,5,4,4,5,4,5,5


In [ ]:
# # setiap selesai submit, db harus di close
# conn.close()

# CONNECT & CREATE DATABASE

In [ ]:
# Koneksi ke database setiap mulai sistem
conn = sqlite3.connect("/content/bone_recommender.db")
cursor = conn.cursor()

In [ ]:
cursor.execute("""
create table if not exists mst_tbl(
user_id TEXT PRIMARY KEY,
created_at TEXT,
kategori TEXT,
umur INT,
gender TEXT,
domisili TEXT,


'INN-CE1' INT, 'SE-M1' INT,
'NACH-FF2' INT, 'LOC-I1' INT,
'SE-P2' INT, 'INN-O2' INT,
'SE-IP1' INT, 'NACH-HS1' INT,
'SE-IF1' INT, 'LOC-E2' INT,
'SE-S1' INT, 'INN-W1' INT,
'NACH-HS2' INT, 'SE-IF2' INT,
'INN-CE2' INT, 'LOC-E1' INT,
'SE-M2' INT, 'SE-S2' INT,
'LOC-I2' INT, 'INN-W2' INT,
'SE-P1' INT, 'INN-O1' INT,
'SE-IP2' INT, 'NACH-FF1' INT,


'CON-2' INT, 'OPE-3' INT,
'NEU-1' INT, 'AGR-2' INT,
'EXT-2' INT, 'CON-1' INT,
'OPE-1' INT, 'AGR-1' INT,
'NEU-3' INT, 'EXT-3' INT,
'OPE-2' INT, 'AGR-3' INT,
'NEU-2' INT, 'CON-3' INT,
'EXT-1' INT
)
""")

# BIODATA (INPUT)


In [ ]:
# KATEGORI
kategori_label = widgets.Label(
    "Mohon pilih kategori yang paling menggambarkan peran Anda dalam aktivitas ekonomi atau profesional saat ini:"
)
kategori_input = widgets.Dropdown(
    options=["Calon Wirausahawan", "Pelaku UMKM", "Pelajar/Mahasiswa", "Profesional"],
    layout=widgets.Layout(width='350px')
)
kategori_block = widgets.VBox([kategori_label, kategori_input])

# UMUR
umur_label = widgets.Label("Berapa usia Anda saat ini?")
umur_input = widgets.BoundedIntText(
    value=18,
    min=1,
    max=99,
    layout=widgets.Layout(width='120px')
)
umur_block = widgets.VBox([umur_label, umur_input])

# GENDER
gender_label = widgets.Label("Jenis kelamin Anda:")
gender_input = widgets.Dropdown(
    options=["Laki-laki", "Perempuan"],
    layout=widgets.Layout(width='200px')
)
gender_block = widgets.VBox([gender_label, gender_input])

# DOMISILI
domisili_label = widgets.Label("Dimana domisili Anda saat ini?")

domisili_input = widgets.Combobox(
    options=[
          # ACEH
          "Kabupaten Aceh Barat", "Kabupaten Aceh Barat Daya", "Kabupaten Aceh Besar",
          "Kabupaten Aceh Jaya", "Kabupaten Aceh Selatan", "Kabupaten Aceh Singkil",
          "Kabupaten Aceh Tamiang", "Kabupaten Aceh Tengah", "Kabupaten Aceh Tenggara",
          "Kabupaten Aceh Timur", "Kabupaten Aceh Utara", "Kabupaten Bener Meriah",
          "Kabupaten Bireuen", "Kabupaten Gayo Lues", "Kabupaten Nagan Raya",
          "Kabupaten Pidie", "Kabupaten Pidie Jaya", "Kabupaten Simeulue",
          "Kota Banda Aceh", "Kota Langsa", "Kota Lhokseumawe", "Kota Sabang",
          "Kota Subulussalam",

          # SUMATERA UTARA
          "Kabupaten Asahan", "Kabupaten Batubara", "Kabupaten Dairi", "Kabupaten Deli Serdang",
          "Kabupaten Humbang Hasundutan", "Kabupaten Karo", "Kabupaten Labuhanbatu",
          "Kabupaten Labuhanbatu Selatan", "Kabupaten Labuhanbatu Utara", "Kabupaten Langkat",
          "Kabupaten Mandailing Natal", "Kabupaten Nias", "Kabupaten Nias Barat",
          "Kabupaten Nias Selatan", "Kabupaten Nias Utara", "Kabupaten Padang Lawas",
          "Kabupaten Padang Lawas Utara", "Kabupaten Pakpak Bharat", "Kabupaten Samosir",
          "Kabupaten Serdang Bedagai", "Kabupaten Simalungun", "Kabupaten Tapanuli Selatan",
          "Kabupaten Tapanuli Tengah", "Kabupaten Tapanuli Utara", "Kabupaten Toba",
          "Kota Binjai", "Kota Gunungsitoli", "Kota Medan", "Kota Padangsidimpuan",
          "Kota Pematangsiantar", "Kota Sibolga", "Kota Tanjungbalai", "Kota Tebing Tinggi",

          # SUMATERA BARAT
          "Kabupaten Agam", "Kabupaten Dharmasraya", "Kabupaten Kepulauan Mentawai",
          "Kabupaten Lima Puluh Kota", "Kabupaten Padang Pariaman", "Kabupaten Pasaman",
          "Kabupaten Pasaman Barat", "Kabupaten Pesisir Selatan", "Kabupaten Sijunjung",
          "Kabupaten Solok", "Kabupaten Solok Selatan", "Kabupaten Tanah Datar",
          "Kota Bukittinggi", "Kota Padang", "Kota Padangpanjang", "Kota Pariaman",
          "Kota Payakumbuh", "Kota Sawahlunto", "Kota Solok",

          # RIAU
          "Kabupaten Bengkalis", "Kabupaten Indragiri Hilir", "Kabupaten Indragiri Hulu",
          "Kabupaten Kampar", "Kabupaten Kepulauan Meranti", "Kabupaten Kuantan Singingi",
          "Kabupaten Pelalawan", "Kabupaten Rokan Hilir", "Kabupaten Rokan Hulu",
          "Kabupaten Siak", "Kota Dumai", "Kota Pekanbaru",

          # JAMBI
          "Kabupaten Batanghari", "Kabupaten Bungo", "Kabupaten Kerinci",
          "Kabupaten Merangin", "Kabupaten Muaro Jambi", "Kabupaten Sarolangun",
          "Kabupaten Tanjung Jabung Barat", "Kabupaten Tanjung Jabung Timur",
          "Kabupaten Tebo", "Kota Jambi", "Kota Sungai Penuh",

          # SUMATERA SELATAN
          "Kabupaten Banyuasin", "Kabupaten Empat Lawang", "Kabupaten Lahat",
          "Kabupaten Muara Enim", "Kabupaten Musi Banyuasin", "Kabupaten Musi Rawas",
          "Kabupaten Musi Rawas Utara", "Kabupaten Ogan Ilir", "Kabupaten Ogan Komering Ilir",
          "Kabupaten Ogan Komering Ulu", "Kabupaten Ogan Komering Ulu Selatan",
          "Kabupaten Ogan Komering Ulu Timur", "Kota Lubuklinggau", "Kota Pagar Alam",
          "Kota Palembang", "Kota Prabumulih",

          # BENGKULU
          "Kabupaten Bengkulu Selatan", "Kabupaten Bengkulu Tengah", "Kabupaten Bengkulu Utara",
          "Kabupaten Kaur", "Kabupaten Kepahiang", "Kabupaten Lebong",
          "Kabupaten Mukomuko", "Kabupaten Rejang Lebong", "Kabupaten Seluma",
          "Kota Bengkulu",

          # LAMPUNG
          "Kabupaten Lampung Barat", "Kabupaten Lampung Selatan", "Kabupaten Lampung Tengah",
          "Kabupaten Lampung Timur", "Kabupaten Lampung Utara", "Kabupaten Mesuji",
          "Kabupaten Pesawaran", "Kabupaten Pesisir Barat", "Kabupaten Pringsewu",
          "Kabupaten Tanggamus", "Kabupaten Tulang Bawang", "Kabupaten Tulang Bawang Barat",
          "Kabupaten Way Kanan", "Kota Bandar Lampung", "Kota Metro",

          # KEP. BANGKA BELITUNG
          "Kabupaten Bangka", "Kabupaten Bangka Barat", "Kabupaten Bangka Selatan",
          "Kabupaten Bangka Tengah", "Kabupaten Belitung", "Kabupaten Belitung Timur",
          "Kota Pangkalpinang",

          # KEP. RIAU
          "Kabupaten Bintan", "Kabupaten Karimun", "Kabupaten Kepulauan Anambas",
          "Kabupaten Lingga", "Kabupaten Natuna", "Kota Batam", "Kota Tanjungpinang",

          # DKI JAKARTA
          "Kota Jakarta Barat", "Kota Jakarta Pusat", "Kota Jakarta Selatan",
          "Kota Jakarta Timur", "Kota Jakarta Utara", "Kabupaten Kepulauan Seribu",

          # JAWA BARAT
          "Kabupaten Bandung", "Kabupaten Bandung Barat", "Kabupaten Bekasi",
          "Kabupaten Bogor", "Kabupaten Ciamis", "Kabupaten Cianjur", "Kabupaten Cirebon",
          "Kabupaten Garut", "Kabupaten Indramayu", "Kabupaten Karawang",
          "Kabupaten Kuningan", "Kabupaten Majalengka", "Kabupaten Pangandaran",
          "Kabupaten Purwakarta", "Kabupaten Subang", "Kabupaten Sukabumi",
          "Kabupaten Sumedang", "Kabupaten Tasikmalaya", "Kota Bandung", "Kota Banjar",
          "Kota Bekasi", "Kota Bogor", "Kota Cimahi", "Kota Cirebon", "Kota Depok",
          "Kota Sukabumi", "Kota Tasikmalaya",

          # JAWA TENGAH
          "Kabupaten Banjarnegara", "Kabupaten Banyumas", "Kabupaten Batang",
          "Kabupaten Blora", "Kabupaten Boyolali", "Kabupaten Brebes", "Kabupaten Cilacap",
          "Kabupaten Demak", "Kabupaten Grobogan", "Kabupaten Jepara",
          "Kabupaten Karanganyar", "Kabupaten Kebumen", "Kabupaten Kendal",
          "Kabupaten Klaten", "Kabupaten Kudus", "Kabupaten Magelang",
          "Kabupaten Pati", "Kabupaten Pekalongan", "Kabupaten Pemalang",
          "Kabupaten Purbalingga", "Kabupaten Purworejo", "Kabupaten Rembang",
          "Kabupaten Semarang", "Kabupaten Sragen", "Kabupaten Sukoharjo",
          "Kabupaten Tegal", "Kabupaten Temanggung", "Kabupaten Wonogiri",
          "Kabupaten Wonosobo", "Kota Magelang", "Kota Pekalongan", "Kota Salatiga",
          "Kota Semarang", "Kota Surakarta", "Kota Tegal",

          # DI YOGYAKARTA
          "Kabupaten Bantul", "Kabupaten Gunungkidul", "Kabupaten Kulon Progo",
          "Kabupaten Sleman", "Kota Yogyakarta",

          # JAWA TIMUR
          "Kabupaten Bangkalan", "Kabupaten Banyuwangi", "Kabupaten Blitar",
          "Kabupaten Bojonegoro", "Kabupaten Bondowoso", "Kabupaten Gresik",
          "Kabupaten Jember", "Kabupaten Jombang", "Kabupaten Kediri",
          "Kabupaten Lamongan", "Kabupaten Lumajang", "Kabupaten Madiun",
          "Kabupaten Magetan", "Kabupaten Malang", "Kabupaten Mojokerto",
          "Kabupaten Nganjuk", "Kabupaten Ngawi", "Kabupaten Pacitan",
          "Kabupaten Pamekasan", "Kabupaten Pasuruan", "Kabupaten Ponorogo",
          "Kabupaten Probolinggo", "Kabupaten Sampang", "Kabupaten Sidoarjo",
          "Kabupaten Situbondo", "Kabupaten Sumenep", "Kabupaten Trenggalek",
          "Kabupaten Tuban", "Kabupaten Tulungagung", "Kota Batu", "Kota Blitar",
          "Kota Kediri", "Kota Madiun", "Kota Malang", "Kota Mojokerto",
          "Kota Pasuruan", "Kota Probolinggo", "Kota Surabaya",

          # BANTEN
          "Kabupaten Lebak", "Kabupaten Pandeglang",
          "Kabupaten Serang", "Kabupaten Tangerang",
          "Kota Cilegon", "Kota Serang", "Kota Tangerang",
          "Kota Tangerang Selatan",

          # BALI
          "Kabupaten Badung", "Kabupaten Bangli", "Kabupaten Buleleng",
          "Kabupaten Gianyar", "Kabupaten Jembrana", "Kabupaten Karangasem",
          "Kabupaten Klungkung", "Kabupaten Tabanan", "Kota Denpasar",

          # NUSA TENGGARA BARAT
          "Kabupaten Bima", "Kabupaten Dompu", "Kabupaten Lombok Barat",
          "Kabupaten Lombok Tengah", "Kabupaten Lombok Timur", "Kabupaten Lombok Utara",
          "Kabupaten Sumbawa", "Kabupaten Sumbawa Barat", "Kota Bima", "Kota Mataram",

          # NUSA TENGGARA TIMUR
          "Kabupaten Alor", "Kabupaten Belu", "Kabupaten Ende", "Kabupaten Flores Timur",
          "Kabupaten Kupang", "Kabupaten Lembata", "Kabupaten Malaka", "Kabupaten Manggarai",
          "Kabupaten Manggarai Barat", "Kabupaten Manggarai Timur", "Kabupaten Ngada",
          "Kabupaten Nagekeo", "Kabupaten Rote Ndao", "Kabupaten Sabu Raijua",
          "Kabupaten Sikka", "Kabupaten Sumba Barat", "Kabupaten Sumba Barat Daya",
          "Kabupaten Sumba Tengah", "Kabupaten Sumba Timur", "Kabupaten Timor Tengah Selatan",
          "Kabupaten Timor Tengah Utara", "Kota Kupang",

          # KALIMANTAN BARAT
          "Kabupaten Bengkayang", "Kabupaten Kapuas Hulu", "Kabupaten Kayong Utara",
          "Kabupaten Ketapang", "Kabupaten Kubu Raya", "Kabupaten Landak",
          "Kabupaten Melawi", "Kabupaten Mempawah", "Kabupaten Sambas",
          "Kabupaten Sanggau", "Kabupaten Sekadau", "Kabupaten Sintang",
          "Kota Pontianak", "Kota Singkawang",

          # KALIMANTAN TENGAH
          "Kabupaten Barito Selatan", "Kabupaten Barito Timur", "Kabupaten Barito Utara",
          "Kabupaten Gunung Mas", "Kabupaten Kapuas", "Kabupaten Katingan",
          "Kabupaten Kotawaringin Barat", "Kabupaten Kotawaringin Timur",
          "Kabupaten Lamandau", "Kabupaten Murung Raya", "Kabupaten Pulang Pisau",
          "Kabupaten Seruyan", "Kabupaten Sukamara", "Kota Palangka Raya",

          # KALIMANTAN SELATAN
          "Kabupaten Balangan", "Kabupaten Banjar", "Kabupaten Barito Kuala",
          "Kabupaten Hulu Sungai Selatan", "Kabupaten Hulu Sungai Tengah",
          "Kabupaten Hulu Sungai Utara", "Kabupaten Kotabaru", "Kabupaten Tabalong",
          "Kabupaten Tanah Bumbu", "Kabupaten Tanah Laut", "Kabupaten Tapin",
          "Kota Banjarbaru", "Kota Banjarmasin",

          # KALIMANTAN TIMUR
          "Kabupaten Berau", "Kabupaten Kutai Barat", "Kabupaten Kutai Kartanegara",
          "Kabupaten Kutai Timur", "Kabupaten Mahakam Ulu", "Kabupaten Paser",
          "Kabupaten Penajam Paser Utara", "Kota Balikpapan", "Kota Bontang",
          "Kota Samarinda",

          # KALIMANTAN UTARA
          "Kabupaten Bulungan", "Kabupaten Malinau", "Kabupaten Nunukan",
          "Kabupaten Tana Tidung", "Kota Tarakan",

          # SULAWESI UTARA
          "Kabupaten Bolaang Mongondow", "Kabupaten Bolaang Mongondow Selatan",
          "Kabupaten Bolaang Mongondow Timur", "Kabupaten Bolaang Mongondow Utara",
          "Kabupaten Kepulauan Sangihe", "Kabupaten Kepulauan Siau Tagulandang Biaro",
          "Kabupaten Kepulauan Talaud", "Kabupaten Minahasa", "Kabupaten Minahasa Selatan",
          "Kabupaten Minahasa Tenggara", "Kabupaten Minahasa Utara",
          "Kota Bitung", "Kota Kotamobagu", "Kota Manado", "Kota Tomohon",

          # SULAWESI TENGAH
          "Kabupaten Banggai", "Kabupaten Banggai Kepulauan", "Kabupaten Banggai Laut",
          "Kabupaten Buol", "Kabupaten Donggala", "Kabupaten Morowali",
          "Kabupaten Morowali Utara", "Kabupaten Parigi Moutong", "Kabupaten Poso",
          "Kabupaten Sigi", "Kabupaten Tojo Una-Una", "Kabupaten Tolitoli",
          "Kota Palu",

          # SULAWESI SELATAN
          "Kabupaten Bantaeng", "Kabupaten Barru", "Kabupaten Bone", "Kabupaten Bulukumba",
          "Kabupaten Enrekang", "Kabupaten Gowa", "Kabupaten Jeneponto",
          "Kabupaten Kepulauan Selayar", "Kabupaten Luwu", "Kabupaten Luwu Timur",
          "Kabupaten Luwu Utara", "Kabupaten Maros", "Kabupaten Pangkajene Kepulauan",
          "Kabupaten Pinrang", "Kabupaten Sidenreng Rappang", "Kabupaten Sinjai",
          "Kabupaten Soppeng", "Kabupaten Takalar", "Kabupaten Tana Toraja",
          "Kabupaten Toraja Utara", "Kota Makassar", "Kota Palopo", "Kota Parepare",

          # SULAWESI TENGGARA
          "Kabupaten Bombana", "Kabupaten Buton", "Kabupaten Buton Selatan",
          "Kabupaten Buton Tengah", "Kabupaten Buton Utara", "Kabupaten Kolaka",
          "Kabupaten Kolaka Timur", "Kabupaten Kolaka Utara", "Kabupaten Konawe",
          "Kabupaten Konawe Kepulauan", "Kabupaten Konawe Selatan",
          "Kabupaten Konawe Utara", "Kabupaten Muna", "Kabupaten Muna Barat",
          "Kabupaten Wakatobi", "Kota Baubau", "Kota Kendari",

          # GORONTALO
          "Kabupaten Boalemo", "Kabupaten Bone Bolango", "Kabupaten Gorontalo",
          "Kabupaten Gorontalo Utara", "Kabupaten Pohuwato", "Kota Gorontalo",

          # SULAWESI BARAT
          "Kabupaten Majene", "Kabupaten Mamasa", "Kabupaten Mamuju",
          "Kabupaten Mamuju Tengah", "Kabupaten Pasangkayu", "Kabupaten Polewali Mandar",

          # MALUKU
          "Kabupaten Buru", "Kabupaten Buru Selatan", "Kabupaten Kepulauan Aru",
          "Kabupaten Maluku Barat Daya", "Kabupaten Maluku Tengah",
          "Kabupaten Maluku Tenggara", "Kabupaten Seram Bagian Barat",
          "Kabupaten Seram Bagian Timur", "Kota Ambon", "Kota Tual",

          # MALUKU UTARA
          "Kabupaten Halmahera Barat", "Kabupaten Halmahera Tengah",
          "Kabupaten Halmahera Timur", "Kabupaten Halmahera Selatan",
          "Kabupaten Halmahera Utara", "Kabupaten Kepulauan Sula",
          "Kabupaten Pulau Morotai", "Kabupaten Pulau Taliabu",
          "Kota Ternate", "Kota Tidore Kepulauan",

          # PAPUA (lama)
          "Kabupaten Asmat", "Kabupaten Biak Numfor", "Kabupaten Boven Digoel",
          "Kabupaten Deiyai", "Kabupaten Dogiyai", "Kabupaten Intan Jaya",
          "Kabupaten Jayapura", "Kabupaten Jayawijaya", "Kabupaten Keerom",
          "Kabupaten Lanny Jaya", "Kabupaten Mamberamo Raya", "Kabupaten Mamberamo Tengah",
          "Kabupaten Mappi", "Kabupaten Merauke", "Kabupaten Mimika",
          "Kabupaten Nabire", "Kabupaten Nduga", "Kabupaten Paniai",
          "Kabupaten Pegunungan Bintang", "Kabupaten Puncak", "Kabupaten Puncak Jaya",
          "Kabupaten Sarmi", "Kabupaten Supiori", "Kabupaten Tolikara",
          "Kabupaten Waropen", "Kota Jayapura",

          # PAPUA TENGAH
          "Kabupaten Puncak", "Kabupaten Paniai", "Kabupaten Dogiyai", "Kabupaten Deiyai",
          "Kabupaten Nabire", "Kabupaten Mimika",

          # PAPUA PEGUNUNGAN
          "Kabupaten Jayawijaya", "Kabupaten Lanny Jaya", "Kabupaten Mamberamo Tengah",
          "Kabupaten Nduga", "Kabupaten Tolikara", "Kabupaten Yahukimo",
          "Kabupaten Yalimo", "Kabupaten Pegunungan Bintang",

          # PAPUA SELATAN
          "Kabupaten Merauke", "Kabupaten Mappi", "Kabupaten Asmat",
          "Kabupaten Boven Digoel",

          # PAPUA BARAT
          "Kabupaten Fakfak", "Kabupaten Kaimana", "Kabupaten Manokwari",
          "Kabupaten Manokwari Selatan", "Kabupaten Pegunungan Arfak",
          "Kabupaten Teluk Bintuni", "Kabupaten Teluk Wondama",
          "Kota Sorong",

          # PAPUA BARAT DAYA (baru)
          "Kabupaten Sorong", "Kabupaten Sorong Selatan", "Kabupaten Tambrauw",
          "Kabupaten Maybrat", "Kabupaten Raja Ampat", "Kota Sorong"
      ],
    placeholder='Ketik untuk mencari atau masukkan sendiri...',
    ensure_option=False,             # supaya input manual tetap diterima
    layout=widgets.Layout(width='350px')
)

# Gabungkan tampilan
domisili_block = widgets.VBox([domisili_label, domisili_input])

In [ ]:
display(kategori_block, umur_block, gender_block, domisili_block)

# DOT FUNCTION (UI KUESIONER)

In [ ]:
def likert_item():
    left_label = widgets.Label("Sangat Tidak Setuju", layout=widgets.Layout(width="150px"))
    right_label = widgets.Label("Sangat Setuju", layout=widgets.Layout(width="150px"))

    scale = widgets.ToggleButtons(
        options=[1, 2, 3, 4, 5],
        button_style='',   # default
        tooltips=['1','2','3','4','5'],
        layout=widgets.Layout(width='250px')
    )

    # sedikit styling agar kelihatan bulat
    scale.style.button_width = '40px'

    likert_row = widgets.HBox([
        left_label,
        scale,
        right_label
    ])

    return scale, likert_row

# BAGIAN A (INPUT)

In [ ]:
questions_A = [
    "Saya tertantang oleh ketidakpastian dan masalah yang belum terpecahkan. (INN-CE1)",
    "Saya percaya diri dapat melakukan networking, yaitu membangun hubungan dan bertukar informasi yang bermanfaat dengan berbagai pihak untuk mendukung kegiatan atau proyek saya. (SE-M1)",
    "Jika saya tidak langsung mengerti sebuah masalah, saya mulai merasa cemas. (NACH-FF2)",
    "Jika saya bekerja dengan sungguh-sungguh, saya bisa mencapai hasil yang saya inginkan. (LOC-I1)",
    "Saya percaya diri dapat memperkirakan jumlah modal awal dan modal kerja yang diperlukan untuk memulai atau menjalankan usaha, proyek, atau kegiatan baru. (SE-P2)",
    "Saya terbuka untuk menggunakan cara baru meskipun belum umum dipraktikkan orang lain. (INN-O2)",
    "Saya percaya diri dapat memotivasi dan mendorong anggota tim agar semangat dalam bekerja. (SE-IP1)",
    "Saya termotivasi untuk segera bertindak ketika menghadapi tantangan yang bisa saya selesaikan. (NACH-HS1)",
    "Saya percaya diri dapat mengorganisir dan memelihara catatan keuangan untuk usaha, proyek, atau kegiatan yang saya jalankan. (SE-IF1)",
    "Saya percaya bahwa hasil kerja saya banyak dipengaruhi oleh keadaan atau orang lain, bukan sepenuhnya oleh usaha saya. (LOC-E2)",
    "Saya percaya diri dapat mengidentifikasi kebutuhan atau peluang baru yang bisa diwujudkan menjadi produk, layanan, atau solusi. (SE-S1)",
    "Saya bersedia untuk mencoba cara baru meskipun berbeda dari kebiasaan saya sebelumnya. (INN-W1)",
    "Saya menikmati situasi di mana saya bisa menggunakan dan mengembangkan kemampuan saya. (NACH-HS2)",
    "Saya percaya diri dapat mengelola aset atau sumber daya keuangan secara efektif dalam usaha, proyek, atau kegiatan saya. (SE-IF2)",
    "Saya sering mengimprovisasi metode untuk memecahkan masalah ketika jawabannya tidak jelas. (INN-CE2)",
    "Saya merasa keberhasilan dalam pekerjaan banyak ditentukan oleh faktor di luar kendali saya. (LOC-E1)",
    "Saya percaya diri dapat meyakinkan orang lain untuk memahami, mendukung, dan ikut berkomitmen pada visi serta rencana yang saya buat. (SE-M2)",
    "Saya percaya diri dapat merancang ide atau solusi baru yang sesuai dengan kebutuhan pasar. (SE-S2)",
    "Saya percaya bahwa hasil pekerjaan saya bergantung pada usaha dan cara saya bekerja. (LOC-I2)",
    "Saya merasa antusias ketika harus beradaptasi dengan situasi baru. (INN-W2)",
    "Saya percaya diri dapat menyusun rencana bisnis, termasuk memperkirakan permintaan pasar dan strategi pemasaran. (SE-P1)",
    "Saya menikmati mencoba ide-ide baru. (INN-O1)",
    "Saya percaya diri dapat memilih orang yang tepat untuk bekerja sama dan membantu mereka mengembangkan kemampuan yang dibutuhkan. (SE-IP2)",
    "Saya takut gagal dalam situasi agak sulit, ketika banyak hal bergantung pada saya. (NACH-FF1)"
    ]

In [ ]:
likert_widgets_A = []
ui_blocks = []

for idx, q in enumerate(questions_A):
    label = widgets.HTML(f"<b>{idx+1}. {q}</b>")
    scale_widget, ui_row = likert_item()
    likert_widgets_A.append(scale_widget)
    ui_blocks.append(widgets.VBox([label, ui_row]))

display(widgets.VBox(ui_blocks))

# BAGIAN B (INPUT)

In [ ]:
questions_B = [
    "Saya adalah seseorang yang cenderung malas (atau agak malas). (CON-2)",
    "Saya adalah seseorang yang memiliki imajinasi yang jelas/hidup (atau kaya) dan penuh fantasi. (OPE-3)",
    "Saya adalah seseorang yang sering khawatir. (NEU-1)",
    "Saya adalah seseorang yang mudah memaafkan (atau bisa memaafkan). (AGR-2)",
    "Saya adalah seseorang yang bisa keluar sendiri dan bersosialisasi (atau mudah bergaul). (EXT-2)",
    "Saya adalah seseorang yang bekerja secara teliti (atau menyeluruh). (CON-1)",
    "Saya adalah seseorang yang orisinil dan membawa ide-ide baru. (OPE-1)",
    "Saya adalah seseorang yang kadang-kadang agak kasar (atau sedikit tidak sopan) kepada orang lain. (AGR-1)",
    "Saya adalah seseorang yang santai dan dapat mengatasi stres dengan baik. (NEU-3)",
    "Saya adalah seseorang yang pendiam (atau tertutup). (EXT-3)",
    "Saya adalah seseorang yang menghargai pengalaman artistik. (OPE-2)",
    "Saya adalah seseorang yang penuh perhatian dan baik hati terhadap orang lain. (AGR-3)",
    "Saya adalah seseorang yang mudah gugup. (NEU-2)",
    "Saya adalah seseorang yang menyelesaikan tugas secara efektif dan efisien. (CON-3)",
    "Saya adalah seseorang yang komunikatif dan banyak bicara (atau cerewet). (EXT-1)"
    ]

In [ ]:
likert_widgets_B = []
ui_blocks = []

for idx, q in enumerate(questions_B):
    label = widgets.HTML(f"<b>{idx+1}. {q}</b>")
    scale_widget, ui_row = likert_item()
    likert_widgets_B.append(scale_widget)
    ui_blocks.append(widgets.VBox([label, ui_row]))

display(widgets.VBox(ui_blocks))

# INSERT DATABASE

In [ ]:
def save_all(btn):
    user_id = str(uuid.uuid4())
    kategori = kategori_input.value
    umur = umur_input.value
    gender = gender_input.value
    domisili = domisili_input.value
    created_at = datetime.now()

    answers_A = [w.value for w in likert_widgets_A]
    answers_B = [w.value for w in likert_widgets_B]

    # Kolom QA (DI-ESCAPE)
    qa_columns = [
        '[INN-CE1]', '[SE-M1]', '[NACH-FF2]', '[LOC-I1]',
        '[SE-P2]', '[INN-O2]', '[SE-IP1]', '[NACH-HS1]',
        '[SE-IF1]', '[LOC-E2]', '[SE-S1]', '[INN-W1]',
        '[NACH-HS2]', '[SE-IF2]', '[INN-CE2]', '[LOC-E1]',
        '[SE-M2]', '[SE-S2]', '[LOC-I2]', '[INN-W2]',
        '[SE-P1]', '[INN-O1]', '[SE-IP2]', '[NACH-FF1]'
    ]

    # Kolom QB (DI-ESCAPE)
    qb_columns = [
        '[CON-2]', '[OPE-3]', '[NEU-1]', '[AGR-2]',
        '[EXT-2]', '[CON-1]', '[OPE-1]', '[AGR-1]',
        '[NEU-3]', '[EXT-3]', '[OPE-2]', '[AGR-3]',
        '[NEU-2]', '[CON-3]', '[EXT-1]'
    ]

    cursor.execute(f"""
        INSERT INTO mst_tbl(
            user_id, created_at, kategori, umur, gender, domisili,
            {', '.join(qa_columns)},
            {', '.join(qb_columns)}
        )
        VALUES(
            ?, ?, ?, ?, ?, ?,
            {', '.join(['?' for _ in qa_columns])},
            {', '.join(['?' for _ in qb_columns])}
        )
    """, (
        user_id, created_at, kategori, umur, gender, domisili,
        *answers_A, *answers_B
    ))

    conn.commit()
    conn.close()

    print("Data berhasil disimpan!")
    print("user_id:", user_id)


In [ ]:
submit_all = widgets.Button(
    description="Simpan Semua Jawaban",
    button_style="success",
    layout=widgets.Layout(width='250px')
)
submit_all.on_click(save_all)

display(submit_all)

Button(button_style='success', description='Simpan Semua Jawaban', layout=Layout(width='250px'), style=ButtonS…

In [ ]:
# Koneksi ke database setiap mulai sistem
conn = sqlite3.connect("/content/bone_recommender.db")
cursor = conn.cursor()

# BAGIAN A (CONSTRAINT FILTERING)

In [ ]:
# 1. LOAD DATABASE
db_path = "/content/bone_recommender.db"
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM mst_tbl", conn)

In [ ]:
# 2. DOMAIN MAPPING (qA) -> dictionary
domain_cols = {
    "innovativeness": [
        'INN-CE1', 'INN-O2', 'INN-W1', 'INN-CE2', 'INN-W2', 'INN-O1'
    ],
    "self_efficacy": [
        'SE-M1', 'SE-P2', 'SE-IP1', 'SE-IF1', 'SE-S1',
        'SE-IF2', 'SE-M2', 'SE-S2', 'SE-P1', 'SE-IP2'
    ],
    "need_achievement": [
        'NACH-FF2', 'NACH-HS1', 'NACH-HS2', 'NACH-FF1'
    ],
    "loc_internal": ['LOC-I1', 'LOC-I2'],
    "loc_external": ['LOC-E2', 'LOC-E1']
}

In [ ]:
# Hitung rata-rata tiap domain
for domain, cols in domain_cols.items():
    df[f"avg_{domain}"] = df[cols].mean(axis=1)

In [ ]:
# 3. KATEGORISASI DOMAIN KE HIGH/LOW LEVEL
def kategori_score(x):
    if x < 2.5:
        return "low"
    elif 2.5 <= x < 3.5:
        return "mid-low"
    elif 3.5 <= x < 4.25:
        return "mid-high"
    else:
        return "high"

def kategori_loc(internal, external):
    return "internal" if internal >= external else "external"

In [ ]:
df['cat_innovativeness'] = df['avg_innovativeness'].apply(kategori_score)
df['cat_self_efficacy'] = df['avg_self_efficacy'].apply(kategori_score)
df['cat_need_achievement'] = df['avg_need_achievement'].apply(kategori_score)
df['cat_loc'] = df.apply(lambda r: kategori_loc(r['avg_loc_internal'], r['avg_loc_external']), axis=1)

In [ ]:
df[['cat_innovativeness', 'cat_self_efficacy', 'cat_need_achievement', 'cat_loc']]

,cat_innovativeness,cat_self_efficacy,cat_need_achievement,cat_loc


## RULE-MAPPING AWAL SECTOR

In [ ]:
# 4. RULE MAPPING PER LEVEL DOMAIN qA → SEKTOR
rules = {

    "self_efficacy": {
        "high": ["consumer oriented services", "construction", "non-high tech manufacturing", "software", "other business services, cutting-edge technology manufacturing", "high technology manufacturing"],
        "mid-high": ["skill-intensive services"],
        "mid-low": ["wholesale and retail market", "technology intensive services"],
        "low": []
    },

    "innovativeness": {
        "high": ["non-high tech manufacturing", "high technology manufacturing", "software", "technology intensive services", "cutting-edge technology manufacturing"],
        "mid-high": ["consumer oriented services"],
        "mid-low": ["construction", "wholesale and retail market", "skill-intensive services", "other business services"],
        "low": []
    },

    "need_achievement": {
        "high": ["consumer oriented services", "skill-intensive services", "software", "technology intensive services", "cutting-edge technology manufacturing", "high technology manufacturing"],
        "mid-high": ["construction", "non-high tech manufacturing", "other business services"],
        "mid-low": [],
        "low": ["wholesale and retail market"]
    },

    "loc": {
        "internal": ["consumer oriented services", "non-high tech manufacturing", "high technology manufacturing", "software", "technology intensive services", "skill-intensive services", "other business services", "cutting edge technology manufacturing"],
        "external": ["construction", "wholesale and retail market"]
    }
}

In [ ]:
# 5. FUZZY MATCHING — AMBIL SEMUA SEKTOR YANG COMPATIBLE DENGAN RULE MAPPING BY LEVEL DOMAIN

def rekomendasi_per_domain(row):

    # buat set kosong untuk nampung sektor
    # set dipakai agar tidak ada duplikasi
    sektor = set()

    # mengambil kategori domain dari user tersebut
    se = row["cat_self_efficacy"]
    inn = row["cat_innovativeness"]
    nach = row["cat_need_achievement"]
    loc = row["cat_loc"]

    # untuk setiap domain, masukkan sektor-sektor yang match ke dalam set
    # karena pakai set.update(), maka tidak ada duplikasi, bisa multi-source
    sektor.update(rules["self_efficacy"][se])
    sektor.update(rules["innovativeness"][inn])
    sektor.update(rules["need_achievement"][nach])
    sektor.update(rules["loc"][loc])

    # return semua sektor cocok dalam bentuk string, dipisahkan dengan koma
    # sorted() agar urut dan lebih rapi
    return ", ".join(sorted(sektor))

df["rekomendasi_sektor_qA"] = df.apply(rekomendasi_per_domain, axis=1)

ValueError: Cannot set a DataFrame with multiple columns to the single column rekomendasi_sektor_qA

In [ ]:
df[['user_id', 'cat_innovativeness', 'cat_self_efficacy', 'cat_need_achievement', 'cat_loc', 'rekomendasi_sektor_qA']]

In [ ]:
# 6. KONVERSI KATEGORI → ANGKA UNTUK EUCLIDEAN

# mapping kategori ke angka
# konversi level domain menjadi angka 0-3
score_map = {"low": 0, "mid-low": 1, "mid-high": 2, "high": 3}
# loc hanya ada 2 kategori -> 0 dan 1
loc_map = {"external": 0, "internal": 1}

# MEMBUAT USER VECTOR
df["score_SE"] = df["cat_self_efficacy"].map(score_map)
df["score_INN"] = df["cat_innovativeness"].map(score_map)
df["score_NACH"] = df["cat_need_achievement"].map(score_map)
df["score_LOC"] = df["cat_loc"].map(loc_map)

In [ ]:
df[['user_id', 'cat_innovativeness', 'cat_self_efficacy', 'cat_need_achievement', 'cat_loc', 'rekomendasi_sektor_qA', 'score_SE', 'score_INN', 'score_NACH', 'score_LOC']]

## RULE-MAPPING SECTOR DETAIL

In [ ]:
# 7. SEKTOR PROTOTYPE (VERSI NUMERIC)
# vektor ini mewakili profil domain ideal untuk setiap sektor
sector_proto = {
    "consumer oriented services":           [3, 2, 3, 1],
    "construction":                        [3, 1, 2, 0],
    "wholesale and retail market":         [1, 1, 0, 0],
    "non high tech manufacturing":         [3, 3, 2, 1],
    "skill-intensive services":            [2, 1, 3, 1],
    "other business oriented services":    [3, 1, 2, 1],
    "high technology manufacturing":       [3, 3, 3, 1],
    "software":                            [3, 3, 3, 1],
    "technology intensive services":       [1, 3, 3, 1],
    "cutting edge technology manufacturing":[3, 3, 3, 1]
}

In [ ]:
# 8. EUCLIDEAN TIE BREAKER
# setelah di fuzzy matching (hasil tidak "satu pilihan final", tapi beberapa sektor yang mirip/sesuai) menghasilkan beberapa sektor,
# kita perlu pilih yang paling cocok
# inilah gunakanya pakai tie breaker dengan Euclian Distance



def euclidean_distance(user_vec, sector_vec):
  # outputnya: semakin kecil jarak, semakin mirip profil user dengan sektor tersebut

    return np.sqrt(np.sum((np.array(user_vec) - np.array(sector_vec))**2))


def final_sector(row):
    sektor_list = row["rekomendasi_sektor_qA"].split(", ")
    # membaca isi kolom rekomendasi_sektor
    # memecah string menjadi list sektor
    # contoh:
    # "software, consumer oriented services" -> ["software", "consumer oriented services"]

    # kalau cuma 1 sektor, langsung return list berisi 1 value
    if len(sektor_list) == 1:
        return sektor_list

    # buat vektor profil user
    user_vec = [
        row["score_SE"],
        row["score_INN"],
        row["score_NACH"],
        row["score_LOC"]
    ]

    # siapkan dict untuk menyimpan jarak
    jarak_dict = {}

    # hitung jarak untuk setiap sektor
      # baris demi baris:
      # loop semua sektor fuzzy
      # cek apakah sektor punya prototype numeric
      # hitung jaraknya
      # simpan ke dictionary
    for s in sektor_list:
        if s in sector_proto:
            dist = euclidean_distance(user_vec, sector_proto[s])
            jarak_dict[s] = dist

    # cari jarak minimum
    min_dist = min(jarak_dict.values())

    # ambil semua sektor yang jaraknya sama dengan min_dist
    sektor_terbaik = [s for s, d in jarak_dict.items() if d == min_dist]

    return sektor_terbaik


df["final_sector_qA"] = df.apply(final_sector, axis=1)

In [ ]:
def calculate_distances(row):
    sektor_list = row["rekomendasi_sektor_qA"].split(", ")
    user_vec = [
        row["score_SE"],
        row["score_INN"],
        row["score_NACH"],
        row["score_LOC"]
    ]

    jarak_dict = {}

    for s in sektor_list:
        if s in sector_proto:
            dist = euclidean_distance(user_vec, sector_proto[s])
            jarak_dict[s] = dist

    # urutkan dari jarak terkecil (paling mirip) ke terbesar
    jarak_sorted = dict(sorted(jarak_dict.items(), key=lambda x: x[1]))

    return jarak_sorted

In [ ]:
df["euclid_ranking_qA"] = df.apply(calculate_distances, axis=1)

df["top3_euclid_qA"] = df["euclid_ranking_qA"].apply(
    lambda d: [k for k, v in list(d.items())[:3]]
)

df["top5_euclid_qA"] = df["euclid_ranking_qA"].apply(
    lambda d: [k for k, v in list(d.items())[:5]]
)

In [ ]:
df[["rekomendasi_sektor_qA", "euclid_ranking_qA", "top3_euclid_qA", "top5_euclid_qA", "final_sector_qA"]]


## CONTENT-BASED FILTERING

In [ ]:
likert_text_map = {
    "SE-S1": { # Self-Efficacy A. Searching (yakin bisa menemukan peluang usaha)
        1: "Saya sangat tidak percaya diri dalam mengidentifikasi kebutuhan atau peluang baru untuk dijadikan produk, layanan, atau solusi.",
        2: "Saya tidak percaya diri dapat mengidentifikasi kebutuhan atau peluang baru yang bisa dikembangkan menjadi produk, layanan, atau solusi.",
        3: "Saya cukup percaya diri, namun belum konsisten, dalam mengidentifikasi kebutuhan atau peluang baru untuk diwujudkan menjadi produk, layanan, atau solusi.",
        4: "Saya percaya diri dalam mengidentifikasi kebutuhan atau peluang baru yang dapat dikembangkan menjadi produk, layanan, atau solusi.",
        5: "Saya sangat percaya diri dan merasa mampu secara konsisten mengidentifikasi kebutuhan atau peluang baru yang bisa diwujudkan menjadi produk, layanan, atau solusi."
    },
    "SE-S2": { # Self-Efficacy A. Searching (merancang ide/solusi baru)
        1: "Saya sangat tidak percaya diri dalam merancang ide atau solusi baru yang sesuai dengan kebutuhan pasar.",
        2: "Saya tidak percaya diri dapat merancang ide atau solusi baru yang sesuai dengan kebutuhan pasar.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam merancang ide atau solusi baru yang sesuai dengan kebutuhan pasar.",
        4: "Saya percaya diri dalam merancang ide atau solusi baru yang sesuai dengan kebutuhan pasar.",
        5: "Saya sangat percaya diri dan merasa mampu secara konsisten merancang ide atau solusi baru yang sesuai dengan kebutuhan pasar."
    },
    "SE-P1": { # Self-Efficacy B. Planning (menyusun rencana bisnis)
        1: "Saya sangat tidak percaya diri dalam menyusun rencana bisnis, termasuk memperkirakan permintaan pasar dan merancang strategi pemasaran.",
        2: "Saya tidak percaya diri dapat menyusun rencana bisnis, memperkirakan permintaan pasar, atau membuat strategi pemasaran.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam menyusun rencana bisnis serta memperkirakan permintaan pasar dan strategi pemasaran.",
        4: "Saya percaya diri dapat menyusun rencana bisnis, termasuk memperkirakan permintaan pasar dan strategi pemasaran.",
        5: "Saya sangat percaya diri dan merasa mampu secara konsisten menyusun rencana bisnis, memperkirakan permintaan pasar, serta mengembangkan strategi pemasaran."
    },
    "SE-P2": { # Self-Efficacy B. Planning (memperkirakan modal)
        1: "Saya sangat tidak percaya diri dalam memperkirakan jumlah modal awal maupun modal kerja yang dibutuhkan untuk memulai atau menjalankan usaha, proyek, atau kegiatan baru.",
        2: "Saya tidak percaya diri dapat memperkirakan modal awal dan modal kerja yang diperlukan untuk memulai atau menjalankan usaha, proyek, atau kegiatan baru.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam memperkirakan jumlah modal awal dan modal kerja yang diperlukan untuk memulai atau menjalankan usaha, proyek, atau kegiatan baru.",
        4: "Saya percaya diri dapat memperkirakan jumlah modal awal dan modal kerja yang diperlukan untuk memulai atau menjalankan usaha, proyek, atau kegiatan baru.",
        5: "Saya sangat percaya diri dan merasa mampu secara konsisten memperkirakan jumlah modal awal dan modal kerja yang diperlukan untuk memulai atau menjalankan usaha, proyek, atau kegiatan baru."
    },
    "SE-M1": { # Self-Efficacy C. Marshalling (networking)
        1: "Saya sangat tidak percaya diri dalam melakukan networking, termasuk membangun hubungan dan bertukar informasi yang bermanfaat dengan berbagai pihak untuk mendukung kegiatan atau proyek saya.",
        2: "Saya tidak percaya diri dapat melakukan networking atau membangun hubungan yang bermanfaat dengan berbagai pihak untuk mendukung kegiatan atau proyek saya.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam melakukan networking dan membangun hubungan serta bertukar informasi yang bermanfaat dengan berbagai pihak.",
        4: "Saya percaya diri dapat melakukan networking, membangun hubungan, dan bertukar informasi yang bermanfaat dengan berbagai pihak untuk mendukung kegiatan atau proyek saya.",
        5: "Saya sangat percaya diri dan mampu secara konsisten melakukan networking, membangun hubungan, serta bertukar informasi yang bermanfaat dengan berbagai pihak untuk mendukung kegiatan atau proyek saya."
    },
    "SE-M2": { # Self-Efficacy C. Marshalling (meyakinkan orang lain)
        1: "Saya sangat tidak percaya diri dalam meyakinkan orang lain untuk memahami, mendukung, atau berkomitmen pada visi dan rencana yang saya buat.",
        2: "Saya tidak percaya diri dapat meyakinkan orang lain agar memahami, mendukung, dan ikut berkomitmen pada visi serta rencana yang saya buat.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam meyakinkan orang lain untuk memahami, mendukung, dan berkomitmen pada visi serta rencana saya.",
        4: "Saya percaya diri dapat meyakinkan orang lain untuk memahami, mendukung, dan ikut berkomitmen pada visi serta rencana yang saya buat.",
        5: "Saya sangat percaya diri dan mampu secara konsisten meyakinkan orang lain untuk memahami, mendukung, serta berkomitmen pada visi dan rencana yang saya buat."
    },
    "SE-IP1": { # Self-Efficacy D. Implementing-people (memotivasi tim)
        1: "Saya tidak percaya diri bahwa saya mampu memotivasi atau mendorong anggota tim untuk tetap semangat dalam bekerja.",
        2: "Saya kurang percaya diri bahwa saya dapat memotivasi anggota tim agar tetap semangat dalam bekerja.",
        3: "Saya merasa cukup percaya diri, tetapi belum tentu selalu bisa memotivasi anggota tim agar tetap semangat dalam bekerja.",
        4: "Saya percaya diri bahwa saya mampu memotivasi dan mendorong anggota tim untuk tetap semangat dalam bekerja.",
        5: "Saya sangat percaya diri bahwa saya dapat memotivasi dan mendorong anggota tim sehingga mereka tetap bersemangat dalam bekerja."
    },
    "SE-IP2": { # Self-Efficacy D. Implementing-people (memilih orang tepat)
        1: "Saya sangat tidak percaya diri dalam memilih orang yang tepat untuk bekerja sama maupun membantu mereka mengembangkan kemampuan yang dibutuhkan.",
        2: "Saya tidak percaya diri dapat memilih orang yang tepat untuk bekerja sama atau membantu mereka mengembangkan kemampuan yang diperlukan.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam memilih orang yang tepat untuk bekerja sama dan membantu mereka mengembangkan kemampuan yang dibutuhkan.",
        4: "Saya percaya diri dapat memilih orang yang tepat untuk bekerja sama serta membantu mereka mengembangkan kemampuan yang dibutuhkan.",
        5: "Saya sangat percaya diri dan mampu secara konsisten memilih orang yang tepat untuk bekerja sama serta membantu mereka mengembangkan kemampuan yang dibutuhkan."
    },
    "SE-IF1": { # Self-Efficacy E. Implementing-finance (mengorganisir catatan keuangan)
        1: "Saya sangat tidak percaya diri dalam mengorganisir atau memelihara catatan keuangan untuk usaha, proyek, atau kegiatan yang saya jalankan.",
        2: "Saya tidak percaya diri dapat mengorganisir dan memelihara catatan keuangan untuk usaha, proyek, atau kegiatan yang saya jalankan.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam mengorganisir dan memelihara catatan keuangan untuk usaha, proyek, atau kegiatan yang saya jalankan.",
        4: "Saya percaya diri dapat mengorganisir dan memelihara catatan keuangan untuk usaha, proyek, atau kegiatan yang saya jalankan.",
        5: "Saya sangat percaya diri dan mampu secara konsisten mengorganisir serta memelihara catatan keuangan untuk usaha, proyek, atau kegiatan yang saya jalankan."
    },
    "SE-IF2": { # Self-Efficacy E. Implementing-finance (mengelola aset/sumber daya keuangan)
        1: "Saya sangat tidak percaya diri dalam mengelola aset atau sumber daya keuangan secara efektif untuk usaha, proyek, atau kegiatan saya.",
        2: "Saya tidak percaya diri dapat mengelola aset atau sumber daya keuangan secara efektif dalam usaha, proyek, atau kegiatan saya.",
        3: "Saya cukup percaya diri, tetapi belum konsisten, dalam mengelola aset atau sumber daya keuangan secara efektif untuk usaha, proyek, atau kegiatan saya.",
        4: "Saya percaya diri dapat mengelola aset atau sumber daya keuangan secara efektif dalam usaha, proyek, atau kegiatan saya.",
        5: "Saya sangat percaya diri dan mampu secara konsisten mengelola aset atau sumber daya keuangan secara efektif dalam usaha, proyek, atau kegiatan saya."
    },
    "INN-O1": { # Innovativeness A. Openness to new ideas (menikmati ide baru)
        1: "Saya sangat tidak menikmati mencoba ide-ide baru.",
        2: "Saya tidak menikmati mencoba ide-ide baru.",
        3: "Saya kadang menikmati, tetapi tidak selalu, ketika mencoba ide-ide baru.",
        4: "Saya menikmati mencoba ide-ide baru.",
        5: "Saya sangat menikmati dan merasa antusias ketika mencoba ide-ide baru."
    },
    "INN-O2": { # Innovativeness A. Openness to new ideas (terbuka cara baru)
        1: "Saya sangat tidak terbuka untuk menggunakan cara baru, terutama jika cara tersebut belum umum dipraktikkan orang lain.",
        2: "Saya tidak terbuka untuk menggunakan cara baru yang belum umum dipraktikkan orang lain.",
        3: "Saya cukup terbuka, tetapi tidak selalu, untuk menggunakan cara baru meskipun belum umum dipraktikkan orang lain.",
        4: "Saya terbuka untuk menggunakan cara baru meskipun cara tersebut belum umum dipraktikkan orang lain.",
        5: "Saya sangat terbuka dan dengan mudah menerima cara baru meskipun belum umum dipraktikkan orang lain."
    },
    "INN-W1": { # Innovativeness B. Willingness to change (bersedia coba cara baru)
        1: "Saya sangat tidak bersedia mencoba cara baru, terutama jika berbeda dari kebiasaan saya sebelumnya.",
        2: "Saya tidak bersedia mencoba cara baru yang berbeda dari kebiasaan saya sebelumnya.",
        3: "Saya kadang bersedia, tetapi tidak selalu, mencoba cara baru meskipun berbeda dari kebiasaan saya sebelumnya.",
        4: "Saya bersedia mencoba cara baru meskipun berbeda dari kebiasaan saya sebelumnya.",
        5: "Saya sangat bersedia dan nyaman mencoba cara baru meskipun berbeda dari kebiasaan atau pola yang biasa saya lakukan."
    },
    "INN-W2": { # Innovativeness B. Willingness to change (antusias adaptasi)
        1: "Saya sangat tidak merasa antusias ketika harus beradaptasi dengan situasi baru; justru saya cenderung menghindarinya.",
        2: "Saya tidak merasa antusias ketika harus beradaptasi dengan situasi baru.",
        3: "Saya kadang merasa antusias, tetapi tidak selalu, ketika harus beradaptasi dengan situasi baru.",
        4: "Saya merasa antusias ketika harus beradaptasi dengan situasi baru.",
        5: "Saya sangat merasa antusias dan justru bersemangat ketika harus beradaptasi dengan situasi baru."
    },
    "INN-CE1": { # Innovativeness C. Curiosity & experimentation (tertantang ketidakpastian)
        1: "Saya sangat tidak merasa tertantang oleh ketidakpastian atau masalah yang belum terpecahkan; justru saya menghindarinya.",
        2: "Saya tidak merasa tertantang oleh ketidakpastian dan masalah yang belum terpecahkan.",
        3: "Saya kadang merasa tertantang, tetapi tidak selalu, oleh ketidakpastian dan masalah yang belum terpecahkan.",
        4: "Saya merasa tertantang oleh ketidakpastian dan masalah yang belum terpecahkan.",
        5: "Saya sangat merasa tertantang dan justru termotivasi oleh ketidakpastian serta masalah yang belum terpecahkan."
    },
    "INN-CE2": { # Innovativeness C. Curiosity & experimentation (mengimprovisasi metode)
        1: "Saya sangat jarang atau tidak pernah mengimprovisasi metode ketika memecahkan masalah yang jawabannya tidak jelas.",
        2: "Saya jarang mengimprovisasi metode untuk memecahkan masalah ketika jawabannya tidak jelas.",
        3: "Saya kadang-kadang mengimprovisasi metode untuk memecahkan masalah ketika jawabannya tidak jelas.",
        4: "Saya sering mengimprovisasi metode untuk memecahkan masalah ketika jawabannya tidak jelas.",
        5: "Saya sangat sering dan secara alami mengimprovisasi metode untuk memecahkan masalah ketika jawabannya tidak jelas."
    },
    "LOC-I1": { # Locus of Control A. Internal (bekerja sungguh-sungguh)
        1: "Saya merasa bahwa meskipun saya bekerja dengan sungguh-sungguh, saya tetap tidak bisa mencapai hasil yang saya inginkan.",
        2: "Saya menilai bahwa bekerja dengan sungguh-sungguh tidak selalu membuat saya mencapai hasil yang saya inginkan.",
        3: "Saya merasa bekerja dengan sungguh-sungguh kadang membantu, tetapi tidak selalu memastikan saya mendapat hasil yang saya inginkan.",
        4: "Saya meyakini bahwa bekerja dengan sungguh-sungguh biasanya membuat saya mencapai hasil yang saya inginkan.",
        5: "Saya sangat yakin bahwa jika saya bekerja dengan sungguh-sungguh, saya pasti bisa mencapai hasil yang saya inginkan."
    },
    "LOC-I2": { # Locus of Control A. Internal (hasil bergantung pada usaha)
        1: "Saya merasa hasil pekerjaan saya hampir tidak ada hubungannya dengan usaha atau cara saya bekerja.",
        2: "Saya menilai usaha dan cara saya bekerja tidak terlalu menentukan hasil pekerjaan saya.",
        3: "Saya merasa usaha dan cara saya bekerja kadang berpengaruh, kadang tidak terhadap hasil pekerjaan saya.",
        4: "Saya meyakini bahwa usaha dan cara saya bekerja umumnya menentukan hasil pekerjaan saya.",
        5: "Saya sangat yakin bahwa hasil pekerjaan saya sepenuhnya bergantung pada usaha dan cara saya bekerja."
    },
    "LOC-E1": { # Locus of Control B. External (keberhasilan ditentukan faktor luar)
        1: "Saya sangat tidak merasa bahwa keberhasilan dalam pekerjaan ditentukan oleh faktor di luar kendali saya; saya percaya keberhasilan terutama berasal dari usaha dan tindakan saya sendiri.",
        2: "Saya tidak merasa bahwa keberhasilan dalam pekerjaan banyak ditentukan oleh faktor di luar kendali saya.",
        3: "Saya kadang merasa demikian, tetapi tidak selalu; sebagian keberhasilan menurut saya dipengaruhi faktor luar, sebagian dipengaruhi usaha saya.",
        4: "Saya merasa bahwa keberhasilan dalam pekerjaan banyak ditentukan oleh faktor di luar kendali saya.",
        5: "Saya sangat merasa bahwa keberhasilan dalam pekerjaan terutama ditentukan oleh faktor-faktor di luar kendali saya."
    },
    "LOC-E2": { # Locus of Control B. External (hasil kerja dipengaruhi orang lain)
        1: "Saya sangat tidak percaya bahwa hasil kerja saya terutama dipengaruhi oleh keadaan atau orang lain; saya meyakini bahwa usaha saya adalah faktor utamanya.",
        2: "Saya tidak percaya bahwa hasil kerja saya banyak dipengaruhi oleh keadaan atau orang lain.",
        3: "Saya cukup percaya, tetapi tidak sepenuhnya yakin, bahwa hasil kerja saya dipengaruhi oleh keadaan atau orang lain.",
        4: "Saya percaya bahwa hasil kerja saya banyak dipengaruhi oleh keadaan atau orang lain.",
        5: "Saya sangat percaya bahwa hasil kerja saya terutama dipengaruhi oleh keadaan atau orang lain, bukan oleh usaha saya sendiri."
    },
    "NACH-HS1": { # Need for Achievement A. Hope of Success (termotivasi segera bertindak)
        1: "Saya sangat tidak termotivasi untuk segera bertindak ketika menghadapi tantangan, bahkan jika tantangan tersebut bisa saya selesaikan.",
        2: "Saya tidak termotivasi untuk segera bertindak ketika menghadapi tantangan yang bisa saya selesaikan.",
        3: "Saya kadang termotivasi, tetapi tidak selalu, untuk segera bertindak ketika menghadapi tantangan yang bisa saya selesaikan.",
        4: "Saya termotivasi untuk segera bertindak ketika menghadapi tantangan yang bisa saya selesaikan.",
        5: "Saya sangat termotivasi dan langsung terdorong untuk mengambil tindakan ketika menghadapi tantangan yang bisa saya selesaikan."
    },
    "NACH-HS2": { # Need for Achievement A. Hope of Success (menikmati pengembangan kemampuan)
        1: "Saya sangat tidak menikmati situasi di mana saya bisa menggunakan dan mengembangkan kemampuan saya.",
        2: "Saya tidak menikmati situasi di mana saya bisa menggunakan dan mengembangkan kemampuan saya.",
        3: "Saya kadang menikmati, tetapi tidak selalu, situasi di mana saya bisa menggunakan dan mengembangkan kemampuan saya.",
        4: "Saya menikmati situasi di mana saya bisa menggunakan dan mengembangkan kemampuan saya.",
        5: "Saya sangat menikmati dan merasa bersemangat dalam situasi di mana saya dapat menggunakan sekaligus mengembangkan kemampuan saya."
    },
    "NACH-FF1": { # Need for Achievement B. Fear of Failure (takut gagal)
        1: "Saya sangat tidak merasa takut gagal, bahkan dalam situasi sulit ketika banyak hal bergantung pada saya.",
        2: "Saya tidak merasa takut gagal dalam situasi agak sulit ketika banyak hal bergantung pada saya.",
        3: "Saya kadang merasa takut gagal, namun tidak selalu, dalam situasi agak sulit ketika banyak hal bergantung pada saya.",
        4: "Saya merasa takut gagal dalam situasi agak sulit ketika banyak hal bergantung pada saya.",
        5: "Saya sangat merasa takut gagal dalam situasi agak sulit ketika banyak hal bergantung pada saya."
    },
    "NACH-FF2": { # Need for Achievement B. Fear of Failure (merasa cemas)
        1: "Saya sangat tidak merasa cemas ketika tidak langsung mengerti sebuah masalah.",
        2: "Saya tidak merasa cemas meskipun tidak langsung mengerti sebuah masalah.",
        3: "Saya kadang merasa cemas, tetapi tidak selalu, ketika saya tidak langsung mengerti sebuah masalah.",
        4: "Saya merasa cemas ketika saya tidak langsung mengerti sebuah masalah.",
        5: "Saya sangat merasa cemas ketika tidak langsung mengerti sebuah masalah."
    }
}

In [ ]:
def build_user_text(row):
    texts = []
    for col, mapping in likert_text_map.items():
        if col in row:
            texts.append(mapping[int(row[col])])
    return " ".join(texts)

df["user_text_profile"] = df.apply(build_user_text, axis=1)

In [ ]:
df["user_text_profile"]

In [ ]:
  sector_description = {
    "consumer oriented services": "Sektor Anda secara eksklusif berfokus pada interaksi langsung dengan konsumen akhir, mencakup bisnis seperti restoran, salon kecantikan, dan jasa kebersihan. Kunci utama keberhasilan di sini adalah orientasi kuat terhadap kebutuhan pelanggan (customer-oriented) dan kualitas layanan. Karena umumnya terdiri dari usaha skala kecil hingga menengah dengan hambatan masuk yang relatif rendah, persaingan sangat ketat. Keberlangsungan bisnis sangat erat kaitannya dengan kondisi ekonomi domestik dan kecepatan Anda dalam merespons tren gaya hidup serta menjaga daya beli konsumen. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka memiliki keyakinan kuat bahwa hasil pekerjaan mereka sepenuhnya bergantung pada usaha dan cara mereka bekerja, bukan nasib atau kondisi pasar eksternal. ● Dorongan Kerja: Need for Achievement Tinggi. Mereka sangat termotivasi dan langsung terdorong untuk mengambil tindakan ketika menghadapi tantangan dan sangat menikmati situasi di mana mereka dapat menggunakan dan mengembangkan kemampuan mereka. ● Keyakinan Diri: Self-Efficacy Tinggi. Mereka sangat percaya diri dan mampu secara konsisten mengidentifikasi peluang, menyusun rencana bisnis, dan mengelola sumber daya (termasuk networking). ● Adaptasi: Innovativeness Mid-High. Mereka menikmati mencoba ide-ide baru dan terbuka untuk menggunakan cara baru meskipun belum umum.",
    "construction": "Anda beroperasi dalam sektor yang meliputi segala bentuk pembangunan infrastruktur fisik mulai dari gedung, jembatan, hingga kerajinan khusus (tukang ledeng, kontraktor). Sebagai salah satu sektor terbesar, konstruksi bersifat padat modal dan padat kerja. Kinerja proyek Anda sangat dipengaruhi oleh faktor eksternal yang kompleks, seperti fluktuasi cuaca, proses perizinan yang ketat, dan regulasi pemerintah. Oleh karena itu, tantangan utamanya adalah manajemen risiko yang cermat dan memastikan proyek tidak rentan terhadap keterlambatan akibat kondisi lingkungan atau kebijakan. ● Pendorong Hasil: Lokus Kontrol Eksternal. Mereka sangat merasa bahwa keberhasilan dalam pekerjaan terutama ditentukan oleh faktor-faktor di luar kendali mereka, seperti regulasi atau cuaca, dan sangat percaya bahwa hasil kerja mereka banyak dipengaruhi oleh keadaan atau orang lain. ● Dorongan Kerja: Need for Achievement Mid-High. Mereka termotivasi untuk segera bertindak menghadapi tantangan, namun tingkat kecemasan mereka terhadap kegagalan berada pada level moderat. ● Keyakinan Diri: Self-Efficacy Tinggi (Implementasi). Mereka sangat percaya diri pada kemampuan mereka untuk memotivasi dan mendorong anggota tim dan memilih orang yang tepat untuk bekerja sama. ● Adaptasi: Innovativeness Mid-Low. Mereka tidak menikmati mencoba ide-ide baru dan jarang mengimprovisasi metode untuk memecahkan masalah, cenderung mempertahankan kebiasaan kerja yang sudah teruji.",
    "wholesale and retail market": "Sektor Anda merupakan tulang punggung distribusi barang konsumsi, menghubungkan produsen langsung dengan pelanggan akhir melalui saluran perdagangan (dealer mobil, toko grosir, dan e-commerce). Sektor ini adalah cerminan langsung dari kekuatan daya beli masyarakat dan harus merespons dinamika pasar dengan cepat. Dengan adanya kenaikan kelas menengah dan perubahan kebiasaan belanja, Anda dituntut untuk terus beradaptasi dengan tren, meningkatkan efisiensi rantai pasokan, dan menguasai inovasi saluran distribusi modern. ● Pendorong Hasil: Lokus Kontrol Eksternal. Mereka sangat percaya bahwa hasil kerja mereka banyak dipengaruhi oleh keadaan atau orang lain, bukan usaha sendiri. ● Dorongan Kerja: Need for Achievement Rendah. Mereka tidak termotivasi untuk segera bertindak ketika menghadapi tantangan, dan cenderung merasa takut gagal dalam situasi sulit. Mereka tidak menikmati situasi untuk mengembangkan kemampuan. ● Keyakinan Diri: Self-Efficacy Mid-Low. Mereka kurang yakin pada kemampuan mereka untuk memperkirakan modal kerja, tidak percaya diri dalam merancang ide, dan belum konsisten dalam menyusun rencana bisnis. ● Adaptasi: Innovativeness Mid-Low. Mereka tidak terbuka untuk menggunakan cara baru dan jarang mengimprovisasi metode, memilih untuk tetap pada pola dan praktik yang sudah umum.",
    "non high tech manufacturing": "Kategori ini mencakup industri pengolahan dasar yang menghasilkan produk olahan sehari-hari (makanan, tekstil, logam sederhana). Anda beroperasi dengan teknologi produksi yang sudah mapan dan bertujuan menciptakan skala ekonomi serta output manufaktur domestik yang besar. Fokus bisnis utama adalah pada efisiensi proses, kualitas stabil, dan pengurangan biaya produksi, dengan investasi yang relatif rendah dalam riset teknologi mutakhir dibandingkan industri high-tech. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka sangat yakin bahwa jika mereka bekerja dengan sungguh-sungguh, mereka pasti bisa mencapai hasil yang diinginkan. ● Dorongan Kerja: Need for Achievement Mid-High. Mereka termotivasi untuk segera bertindak menghadapi tantangan dan menikmati situasi di mana mereka dapat mengembangkan kemampuan. ● Keyakinan Diri: Self-Efficacy Tinggi. Mereka sangat percaya diri dan mampu secara konsisten mengidentifikasi peluang, menyusun rencana bisnis, dan mengelola keuangan. ● Adaptasi: Innovativeness Tinggi. Mereka sangat inovatif. Mereka sangat menikmati dan antusias ketika mencoba ide-ide baru, sangat bersedia mencoba cara baru, dan secara alami mengimprovisasi metode saat ada masalah.",
    "skill-intensive services": "Sektor Anda berada di domain jasa profesional yang sangat knowledge-intensive (pengacara, konsultan strategi, tenaga ahli). Keberhasilan Anda bergantung pada keahlian khusus dan kompetensi tinggi yang Anda tawarkan kepada klien. Karena Anda menyediakan solusi kompleks dan berperan dalam mendorong pertumbuhan industri lain, tantangan terbesarnya adalah memelihara reputasi, membangun interaksi klien yang kuat, dan terus mengembangkan kompetensi sumber daya manusia yang berpengalaman. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka yakin bahwa hasil pekerjaan mereka sepenuhnya bergantung pada usaha dan cara mereka bekerja. ● Dorongan Kerja: Need for Achievement Tinggi. Mereka sangat termotivasi dan langsung terdorong untuk bertindak ketika menghadapi tantangan, dan sangat menikmati situasi untuk menggunakan dan mengembangkan kemampuan mereka. ● Keyakinan Diri: Self-Efficacy Mid-High. Mereka percaya diri dapat melakukan networking dan meyakinkan orang lain, tetapi belum konsisten atau cukup percaya diri dalam aspek planning atau marshalling sumber daya keuangan yang besar. ● Adaptasi: Innovativeness Mid-Low. Mereka jarang mengimprovisasi metode untuk memecahkan masalah ketika jawabannya tidak jelas, dan tidak terlalu terbuka pada cara baru yang berbeda dari kebiasaan mereka.",
    "other business oriented services": "Anda menyediakan jasa pendukung yang komplementer dan terutama menyasar perusahaan lain (leasing peralatan, keamanan kantor, logistik). Tujuan utama layanan Anda adalah membantu kelancaran operasional bisnis inti klien. Karena sifatnya sebagai pendukung, sektor ini beroperasi dengan standar industri umum dan fokus ketat pada efisiensi, keandalan, dan praktik-praktik terbaik, sehingga jarang memerlukan fokus pada inovasi teknologi yang ekstrim. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka yakin bahwa hasil pekerjaan mereka sepenuhnya bergantung pada usaha dan cara mereka bekerja. ● Dorongan Kerja: Need for Achievement Mid-High. Mereka termotivasi untuk segera bertindak menghadapi tantangan dan menikmati situasi di mana mereka dapat mengembangkan kemampuan. ● Keyakinan Diri: Self-Efficacy Tinggi. Mereka sangat percaya diri dan mampu secara konsisten menyusun rencana bisnis, marshalling sumber daya (termasuk meyakinkan orang lain), dan mengelola aset keuangan. ● Adaptasi: Innovativeness Mid-Low. Mereka tidak menikmati mencoba ide-ide baru dan jarang mengimprovisasi metode. Mereka lebih memilih menjalankan solusi yang sudah pasti dan efisien daripada bereksperimen.",
    "cutting edge technology manufacturing": "Anda adalah bagian dari industri yang memiliki intensitas R&D yang tertinggi (>7%). Fokus bisnis adalah riset dan pengembangan mendalam untuk menghasilkan produk baru yang mendisrupsi pasar (obat biologi, hardware mutakhir). Sektor ini sangat dinamis dan inovatif, menuntut investasi modal dan intelektual yang besar, serta mengandalkan tim yang terdiri dari banyak insinyur dan ilmuwan untuk terus mengembangkan teknologi terbaru. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka sangat yakin bahwa jika mereka bekerja dengan sungguh-sungguh, mereka pasti bisa mencapai hasil yang diinginkan. ● Dorongan Kerja: Need for Achievement Tinggi. Mereka sangat termotivasi dan langsung terdorong untuk bertindak ketika menghadapi tantangan, dan sangat menikmati situasi di mana mereka dapat menggunakan serta mengembangkan kemampuan mereka. ● Keyakinan Diri: Self-Efficacy Tinggi. Mereka sangat percaya diri dan mampu secara konsisten mengidentifikasi peluang, merancang ide, dan mengelola seluruh aspek bisnis, dari tim hingga keuangan. ● Adaptasi: Innovativeness Tinggi. Mereka sangat inovatif. Mereka sangat menikmati ide baru, sangat bersedia mencoba cara berbeda dari kebiasaan, dan merasa tertantang oleh masalah yang belum terpecahkan.",
    "high technology manufacturing": "Sektor Anda memiliki intensitas R&D menengah (2,5–7%). Anda fokus pada pengembangan dan peningkatan produk melalui penggabungan proses produksi canggih dan riset terapan (otomotif, mesin presisi). Meskipun tidak seintensif teknologi mutakhir, Anda secara konsisten mengadopsi inovasi dalam desain dan proses manufaktur untuk memastikan kualitas tinggi dan efisiensi produk yang sudah ada di pasar. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka memiliki keyakinan kuat bahwa hasil pekerjaan mereka sepenuhnya bergantung pada usaha dan cara mereka bekerja. ● Dorongan Kerja: Need for Achievement Tinggi. Mereka sangat termotivasi untuk segera bertindak, sangat menikmati pengembangan kemampuan, dan sangat tidak merasa cemas ketika tidak langsung mengerti sebuah masalah. ● Keyakinan Diri: Self-Efficacy Tinggi. Mereka sangat percaya diri dan mampu secara konsisten menyusun rencana bisnis, memperkirakan modal, dan melakukan networking secara efektif. ● Adaptasi: Innovativeness Tinggi. Mereka sangat inovatif, sangat terbuka untuk menggunakan cara baru, dan sering mengimprovisasi metode untuk memecahkan masalah.",
    "technology intensive services": "Anda beroperasi sebagai penyedia jasa yang fokus pada R&D murni di bidang sains, rekayasa, dan teknologi (firma riset, konsultan teknik). Peran Anda adalah sebagai mitra penelitian yang memiliki kedalaman teknis luar biasa untuk mengaplikasikan teknologi canggih dan menghasilkan solusi kompleks berbasis riset (misalnya, teknologi komunikasi atau pengembangan medis). Sektor ini sangat mengutamakan keahlian dan pengetahuan spesialis. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka yakin bahwa hasil pekerjaan mereka sepenuhnya bergantung pada usaha dan cara mereka bekerja. ● Dorongan Kerja: Need for Achievement Tinggi. Mereka sangat termotivasi untuk segera bertindak dan sangat menikmati situasi untuk menggunakan dan mengembangkan kemampuan mereka. ● Keyakinan Diri: Self-Efficacy Mid-Low. Mereka kurang yakin pada kemampuan mereka untuk memperkirakan modal awal atau tidak percaya diri dapat meyakinkan orang lain untuk berkomitmen pada visi mereka, meskipun mereka memiliki keahlian yang mendalam. ● Adaptasi: Innovativeness Tinggi. Mereka sangat inovatif, sangat antusias ketika harus beradaptasi dengan situasi baru, dan merasa tertantang oleh ketidakpastian.",
    "software": "Sektor Anda berspesialisasi dalam pengembangan produk digital (aplikasi, layanan IT, web design). Industri ini ditandai oleh pertumbuhan yang sangat cepat dan fleksibilitas, karena semua sektor ekonomi membutuhkan solusi perangkat lunak. Keberhasilan Anda bergantung sepenuhnya pada inovasi yang cepat, kemampuan adaptasi terhadap teknologi baru, dan kreativitas untuk memenuhi permintaan pasar digital yang terus berubah. ● Pendorong Hasil: Lokus Kontrol Internal. Mereka sangat yakin bahwa jika mereka bekerja dengan sungguh-sungguh, mereka pasti bisa mencapai hasil yang diinginkan. ● Dorongan Kerja: Need for Achievement Tinggi. Mereka sangat termotivasi dan sangat menikmati situasi untuk mengembangkan kemampuan. Mereka sangat tidak merasa takut gagal dalam situasi sulit. ● Keyakinan Diri: Self-Efficacy Tinggi. Mereka sangat percaya diri dan mampu secara konsisten mengidentifikasi peluang, merancang ide, dan memilih orang yang tepat untuk bekerja sama. ● Adaptasi: Innovativeness Tinggi. Mereka sangat terbuka dan nyaman mencoba cara baru yang berbeda dari kebiasaan, dan sering mengimprovisasi metode untuk memecahkan masalah."
  }

In [ ]:
sector_description

TF-IDF

In [ ]:
def tfidf_similarity(user_text, top5_sectors, sector_desc):

    valid_sectors = [s for s in top5_sectors if s in sector_desc]

    if len(valid_sectors) == 0:
        return {}

    docs = [user_text] + [sector_desc[s] for s in valid_sectors]

    tfidf = TfidfVectorizer()  # TANPA stop_words
    tfidf_matrix = tfidf.fit_transform(docs)

    similarities = cosine_similarity(
        tfidf_matrix[0:1],
        tfidf_matrix[1:]
    )[0]

    return dict(zip(valid_sectors, similarities))

In [ ]:
# TF-IDF itu dicompare berdasarkan hasil filtering 5 sektor dari rule mapping by level dan rule mapping sektor
df["tfidf_similarity"] = df.apply(
    lambda r: tfidf_similarity(
        r["user_text_profile"],
        r["top5_euclid_qA"],
        sector_description
    ),
    axis=1
)

In [ ]:
df["top5_euclid_qA"]

In [ ]:
df["tfidf_similarity"]

In [ ]:
# cek missing
all_sectors = set(sum(df["top5_euclid_qA"].tolist(), []))
missing = all_sectors - set(sector_description.keys())
missing

In [ ]:
df["tfidf_similarity"] = df["tfidf_similarity"].apply(
    lambda d: sorted(d.items(), key=lambda x: x[1], reverse=True)
)

In [ ]:
df["tfidf_similarity"]

In [ ]:
def get_top3_tfidf(tfidf_list):
    if not isinstance(tfidf_list, list) or len(tfidf_list) == 0:
        return []

    # ambil 3 pertama
    return [sector for sector, score in tfidf_list[:3]]

In [ ]:
df["top3_tfidf_sector"] = df["tfidf_similarity"].apply(get_top3_tfidf)

In [ ]:
df["tfidf_similarity"]

In [ ]:
def get_top3_tfidf_with_score(tfidf_list):
    if not isinstance(tfidf_list, list) or len(tfidf_list) == 0:
        return []

    # kalau belum terurut, amankan dengan sort
    sorted_sectors = sorted(tfidf_list, key=lambda x: x[1], reverse=True)

    return sorted_sectors[:3]

In [ ]:
df["top3_tfidf_sector_score"] = df["tfidf_similarity"].apply(get_top3_tfidf_with_score)

In [ ]:
df[["user_id", "top3_tfidf_sector", "top3_tfidf_sector_score"]]

## MAPPING SEKTOR -> CLUSTER

In [ ]:
cluster_mapping = {
    "Adaptive Services": [
        "wholesale and retail market",
        "consumer oriented services",
        "non high tech manufacturing",
        "other business services"
    ],

    "Dynamic Knowledge Innovators": [
        "skill-intensive services",
        "technology intensive services",
        "software",
        "high technology manufacturing"
    ],

    "Strategic Risk Navigators": [
        "technology intensive services",
        "skill-intensive services",
        "other business oriented services"
    ],

    "Multidimensional Service Innovators": [
        "consumer oriented services",
        "skill-intensive services",
        "construction",
        "wholesale and retail market",
        "technology intensive services"
    ],

    "Focused Tech Innovators": [
        "non high tech manufacturing",
        "other business services",
        "software",
        "high technology manufacturing"
    ]
}


In [ ]:
# buat lookup cepat sektor → cluster
sector_to_cluster = {}


# beberapa sektor muncul di lebih dari 1 cluster
# setdefault(s, []) membuat list kosong untuk setiap sektor
# lalu .append(cluster) menambah cluster yang memiliki sektor tersebut
for cluster, sectors in cluster_mapping.items():
    for s in sectors:
        sector_to_cluster.setdefault(s, []).append(cluster)

In [ ]:
# hitung bobot cluster berdasarkan Top5 Euclidean
def cluster_weights_from_top5_qA(top5_list):

    # jika input bukan list atau list kosong → kembalikan dict kosong
    # ini untuk mencegah error jika data tidak valid
    if not isinstance(top5_list, list) or len(top5_list) == 0:
        return {}

    # membuat bobot berdasarkan peringkat (ranking)
    # index 0 = sektor paling mirip → bobot terbesar (5)
    # index 1 → bobot 4, dan seterusnya
    rank_weight = {0:5, 1:4, 2:3, 3:2, 4:1}

    # membuat dictionary bobot awal cluster
    # semua cluster dimulai dari nilai 0
    weights = {cluster: 0 for cluster in cluster_mapping}

    # loop sektor berdasarkan urutan ranking
    for idx, sector in enumerate(top5_list):

        # hanya proses sektor yang punya mapping ke cluster
        if sector in sector_to_cluster:
            # ambil bobot berdasarkan posisi ranking sektor tersebut
            w = rank_weight.get(idx, 0)

            # tambahkan bobot ke setiap cluster yang berisi sektor itu
            # 1 sektor bisa berhubungan ke lebih dari 1 cluster
            for cluster in sector_to_cluster[sector]:
                weights[cluster] += w

    return weights


# SIMPAN hasil bobot cluster
df["cluster_weights_top5_qA"] = df["top3_tfidf_sector"].apply(cluster_weights_from_top5_qA)

In [ ]:
df["cluster_weights_top5_qA"]

In [ ]:
# menyimpan ranking cluster dari bobot (descending)
def sort_cluster_weights(wdict):
    return sorted(wdict.items(), key=lambda x: x[1], reverse=True)

df["cluster_top5_ranking_qA"] = df["cluster_weights_top5_qA"].apply(sort_cluster_weights)

In [ ]:
df[['user_id', 'cluster_top5_ranking_qA', 'cluster_weights_top5_qA']]

## CLUSTER -> BEST SEKTOR

In [ ]:
def top3_clusters_with_sectors_qA(top3_list):

    # jika datanya tidak valid -> kembalikan dict kosong
    if not isinstance(top3_list, list) or len(top3_list) == 0:
        return {}

    # buat dictionary cluster -> list sektor
    hasil = {cluster: [] for cluster in cluster_mapping}

    for sector in top3_list:
        if sector in sector_to_cluster:
            # cek cluster apa saja yg menaungi sektor ini
            clusters = sector_to_cluster[sector]

            # masukkan sektor ke masing-masing cluster tersebut
            for c in clusters:
                hasil[c].append(sector)

    # hapus cluster yang kosong (tidak ada sektor Top 5 di dalamnya)
    hasil = {c: sectors for c, sectors in hasil.items() if len(sectors) > 0}

    return hasil


# simpan ke dataframe
df["top5_cluster_to_sector_qA"] = df["top3_tfidf_sector"].apply(top3_clusters_with_sectors_qA)

In [ ]:
df["top5_cluster_to_sector_qA"]

In [ ]:
def best_cluster_from_weights_qA(wdict):
    # jika kosong → return list kosong
    if not wdict:
        return []

    # cari nilai bobot tertinggi
    max_score = max(wdict.values())

    # ambil semua cluster yang memiliki bobot sama dengan max_score
    best_clusters = [c for c, v in wdict.items() if v == max_score]

    return best_clusters

In [ ]:
df["cluster_top5_best_qA"] = df["cluster_weights_top5_qA"].apply(best_cluster_from_weights_qA)

## OUTPUT BAGIAN A

In [ ]:
df[["user_id", "top3_tfidf_sector","top5_cluster_to_sector_qA", "cluster_weights_top5_qA", "cluster_top5_best_qA"]]

In [ ]:
# df[['user_id','top5_euclid_qA' , 'cluster_top5_ranking_qA', 'cluster_weights_top5_qA', 'top5_cluster_to_sector_qA', 'cluster_top5_best_qA']]

# BAGIAN B (CONSTRAINT FILTERING)

In [ ]:
# dictionary untuk narik data dari database
big5 = {
    "openess": ['OPE-3', 'OPE-1', 'OPE-2'],

    "conscientiousness": {
        "normal": ['CON-1', 'CON-3'],
        "reverse": ['CON-2']
    },

    "extraversion": {
        "normal": ['EXT-2', 'EXT-1'],
        "reverse": ['EXT-3']
    },

    "agreeableness": {
        "normal": ['AGR-2', 'AGR-3'],
        "reverse": ['AGR-1']
    },

    "neuroticism": {
        "normal": ['NEU-1', 'NEU-2'],
        "reverse": ['NEU-3']
    }
}

In [ ]:
def reverse_score(x):
    if pd.isna(x):
        return x
    return 6 - x   # reverse dari skala 1-5

## AVERAGE

Openess

In [ ]:
df["avg_openess"] = df[big5["openess"]].mean(axis=1)

Conscientiousness

In [ ]:
df["con_scored"] = (
    df[big5["conscientiousness"]["normal"]].mean(axis=1) +
    df[big5["conscientiousness"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_conscientiousness"] = df["con_scored"]
df.drop(columns=["con_scored"], inplace=True)

Extraversion

In [ ]:
df["ext_scored"] = (
    df[big5["extraversion"]["normal"]].mean(axis=1) +
    df[big5["extraversion"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_extraversion"] = df["ext_scored"]
df.drop(columns=["ext_scored"], inplace=True)

Agreeableness

In [ ]:
df["agr_scored"] = (
    df[big5["agreeableness"]["normal"]].mean(axis=1) +
    df[big5["agreeableness"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_agreeableness"] = df["agr_scored"]
df.drop(columns=["agr_scored"], inplace=True)

Neuroticism

In [ ]:
df["neu_scored"] = (
    df[big5["neuroticism"]["normal"]].mean(axis=1) +
    df[big5["neuroticism"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_neuroticism"] = df["neu_scored"]
df.drop(columns=["neu_scored"], inplace=True)

In [ ]:
df[["user_id",
    "avg_openess",
    "avg_conscientiousness",
    "avg_extraversion",
    "avg_agreeableness",
    "avg_neuroticism"
]]

## RULE-MAPPING

In [ ]:
def avg_to_notation(x):
    if np.isnan(x):
        return np.nan
    if x >= 4.2:    return '++'
    elif x >= 3.5:  return '+'
    elif x >= 2.5:  return '0'
    elif x >= 1.8:  return '-'
    else:           return '--'

In [ ]:
# buat kolom notasi O, C, E, A, N berdasarkan AVERAGE
df['Note_O'] = df['avg_openess'].apply(avg_to_notation)
df['Note_C'] = df['avg_conscientiousness'].apply(avg_to_notation)
df['Note_E'] = df['avg_extraversion'].apply(avg_to_notation)
df['Note_A'] = df['avg_agreeableness'].apply(avg_to_notation)
df['Note_N'] = df['avg_neuroticism'].apply(avg_to_notation)

In [ ]:
df

In [ ]:
# definisikan rule cluster
clusters = {
    'Adaptive Services': {
        'O':'++', 'C':'0/+', 'E':'+',  'A':'0',    'N':'--'
    },
    'Dynamic Knowledge Innovators': {
        'O':'++', 'C':'-',   'E':'++', 'A':'-',    'N':'--'
    },
    'Strategic Risk Navigators': {
        'O':'++', 'C':'-',   'E':'+',  'A':'-',    'N':'--'
    },
    'Multidimensional Service Innovators': {
        'O':'++','C':'-/+','E':'0/++', 'A':'--/0','N':'--/+'
    },
    'Focused Tech Innovators': {
        'O':'++','C':'-','E':'-/+','A':'0/+','N':'-'
    }
    # ,
    # 'Gender-Related Traits': {
    #     'O':'--/++', 'C':'--/++', 'E':'--/++', 'A':'--/++', 'N':'+/++'
    # }
}

In [ ]:
# fungsi cek kecocokan simbol (mendukung “0/+”, “-/+”, “--/+”)
# intinya cek apakah simbol user cocok dengan simbol cluster
def symbol_matches(resp_sym, cluster_sym):
    if pd.isna(resp_sym):
        return False                # kalau user tidak punya nilai → dianggap tidak cocok
    options = cluster_sym.split('/') # contoh "0/+" → ["0", "+"]
    return resp_sym in options       # cocok kalau simbol user ada di daftar opsi

In [ ]:
# mapping simbol ke angka untuk tie-breaker

symbol_to_num = {'++': 2.0, '+': 1.0, '0': 0.0, '-': -1.0, '--': -2.0} # supaya simbol bisa dipakai dalam perhitungan jarak

# mengubah rule cluster (notasi) -> vektor angka
def cluster_proto_numeric(cluster_sig):
    vals = []  # simpan angka O, C, E, A, N
    for dim in ['O','C','E','A','N']:   # per dimensi
        opts = cluster_sig[dim].split('/')  # contoh "0/+" → ["0","+"]
        nums = [symbol_to_num[o] for o in opts]  # ubah simbol ke angka
        vals.append(np.mean(nums))  # ambil nilai rata-rata
    return np.array(vals)           # kembalikan sebagai vektor numpy

# menghitung prototype vector untuk semua cluster
cluster_protos = {name: cluster_proto_numeric(sig) for name, sig in clusters.items()}

In [ ]:
# # Fungsi assign cluster berbasis matching score + tie breaker
# def assign_cluster_from_avg(row):

#     best_score = -1
#     best_clusters = []

#     # tahap 1 → hitung match per cluster
#     for name, sig in clusters.items():
#         matches = 0
#         for dim in ['O','C','E','A','N']:
#             if symbol_matches(row[f'Note_{dim}'], sig[dim]):
#                 matches += 1
#         score = matches / 5.0

#         if score > best_score:
#             best_score = score
#             best_clusters = [name]
#         elif score == best_score:
#             best_clusters.append(name)

#     # jika hanya satu cluster yang menang
#     if len(best_clusters) == 1:
#         return best_clusters[0], best_score

#     # tahap 2 → tie-breaker berdasarkan jarak euclidean
#     user_vec = np.array([
#         symbol_to_num[row['Note_O']],
#         symbol_to_num[row['Note_C']],
#         symbol_to_num[row['Note_E']],
#         symbol_to_num[row['Note_A']],
#         symbol_to_num[row['Note_N']]
#     ])

#     best_dist = np.inf
#     best_name = None

#     for name in best_clusters:
#         dist = np.linalg.norm(user_vec - cluster_protos[name])
#         if dist < best_dist:
#             best_dist = dist
#             best_name = name

#     return best_name, best_score

In [ ]:
# assignments = df.apply(assign_cluster_from_avg, axis=1)

# df['best_cluster_qB'] = assignments.apply(lambda x: x[0])
# df['best_cluster_qB_Score'] = assignments.apply(lambda x: x[1])

In [ ]:
# df[["user_id", "cluster_top5_best_qA", "best_cluster_qB"]]

In [ ]:
# TOP-3 CLUSTER RANKING FOR QB

# Fungsi: hitung matching score semua cluster (0–1)
def compute_cluster_scores(row):
    cluster_scores = {}

    for name, sig in clusters.items():
        matches = 0
        for dim in ['O','C','E','A','N']:
            if symbol_matches(row[f"Note_{dim}"], sig[dim]):
                matches += 1

        cluster_scores[name] = matches / 5.0

    return cluster_scores


# Fungsi utama dengan tie-breaker + menyimpan top-5 ranking
def assign_cluster_with_top3(row):

    # 1. Hitung matching score semua cluster
    scores = compute_cluster_scores(row)

    # 2. Ranking cluster berdasarkan score (descending)
    sorted_clusters = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Ambil top-3 cluster
    top3_scores = sorted_clusters[:3]
    top3_names = [x[0] for x in top3_scores]

    # 3. Cari cluster pemenang — cek apakah ada tie
    best_score_value = top3_scores[0][1]
    tied_clusters = [c for c, s in top3_scores if s == best_score_value]

    # Jika hanya satu cluster → langsung pemenang
    if len(tied_clusters) == 1:
        best_cluster = tied_clusters[0]
        return best_cluster, best_score_value, top3_names, top3_scores

    # 4. Tie-breaker → Euclidean Distance
    user_vec = np.array([
        symbol_to_num[row['Note_O']],
        symbol_to_num[row['Note_C']],
        symbol_to_num[row['Note_E']],
        symbol_to_num[row['Note_A']],
        symbol_to_num[row['Note_N']]
    ])

    best_dist = np.inf
    best_name = None

    for name in tied_clusters:
        dist = np.linalg.norm(user_vec - cluster_protos[name])
        if dist < best_dist:
            best_dist = dist
            best_name = name

    return best_name, best_score_value, top3_names, top3_scores

kayaknya yang disini jadi top 3 trs top3_cluster_qB akan dilakukan tfidf dan mendapatkan cluster_top5_best_qB (1 cluster terbaik)

In [ ]:
# JALANKAN KE DATAFRAME

results = df.apply(assign_cluster_with_top3, axis=1)

df["top3_clusters_qB"] = results.apply(lambda x: x[2])        # hanya nama cluster
df["top3_clusters_qB_score"] = results.apply(lambda x: x[3])  # tuple (cluster, score)

In [ ]:
df["top3_clusters_qB_score"]

In [ ]:
# def pick_best_clusters_qB_multi(row):
#     top3 = row["top3_clusters_qB_score"]   # list: [(cluster, score), ...]

#     if not isinstance(top3, list) or len(top3) == 0:
#         return ""

#     # Ambil nilai tertinggi
#     highest_score = max(score for _, score in top3)

#     # Jika highest score = 0, kosongkan
#     if highest_score == 0:
#         return ""

#     # Ambil semua cluster yang skor-nya sama dengan nilai tertinggi
#     best_clusters = [name for name, score in top3 if score == highest_score]

#     # Gabungkan dalam 1 string
#     return ", ".join(best_clusters)


# def pick_best_clusters_qB_score(row):
#     top3 = row["top3_clusters_qB_score"]

#     if not isinstance(top3, list) or len(top3) == 0:
#         return np.nan

#     highest_score = max(score for _, score in top3)

#     # Jika highest score = 0, return NaN
#     if highest_score == 0:
#         return np.nan

#     return highest_score

In [ ]:
# # SIMPAN KE DATAFRAME
# df["cluster_top3_best_qB"] = df.apply(pick_best_clusters_qB_multi, axis=1)
# df["cluster_top3_best_qB_score"] = df.apply(pick_best_clusters_qB_score, axis=1)

In [ ]:
df[[
    "user_id",
    "top3_clusters_qB",
    "top3_clusters_qB_score"
    # ,
    # "cluster_top3_best_qB",
    # "cluster_top3_best_qB_score"
]]

## CONTENT-BASED FILTERING

In [ ]:
cluster_descriptions = {
    "Adaptive Services": "Klaster Adaptive Services adalah tipe industri yang unggul karena kemampuannya memadukan kreativitas tinggi dengan stabilitas emosional yang luar biasa, memastikan layanan yang mereka berikan selalu berkualitas dan terukur. Anda dicirikan oleh profil ide yang tertinggi: Anda sangat orisinil dan secara konsisten membawa ide-ide baru. Ini didukung oleh fakta bahwa klaster Anda memiliki imajinasi yang sangat hidup dan kaya, serta sangat menghargai pengalaman artistik. Dalam hal efisiensi, Anda biasanya bekerja dengan teliti dan umumnya memperhatikan detail dengan baik. Anda biasanya menyelesaikan tugas dengan efektif dan efisien; meskipun fleksibel, Anda jarang membiarkan diri sangat cenderung malas. Kekuatan terbesar Anda terletak pada ketenangan, karena klaster ini jarang merasa khawatir dan jarang merasa gugup, menunjukkan bahwa Anda sangat santai dan sangat mampu mengatasi stres dengan baik. Stabilitas ini sangat krusial dalam pengambilan keputusan yang sadar risiko. Secara sosial, Anda biasanya komunikatif dan sering berbicara, dan mudah bergaul, memfasilitasi penyesuaian layanan terhadap kebutuhan pasar. Dalam hubungan, Anda menunjukkan keseimbangan, di mana Anda kadang-kadang bisa bersikap agak kasar namun juga kadang-kadang penuh perhatian dan baik hati.",
    "Dynamic Knowledge Innovators": "Klaster Dynamic Knowledge Innovators adalah mesin inovasi yang paling bersemangat dan cepat, didorong oleh keunggulan teknologi dan semangat kolaborasi yang sangat tinggi, siap mengorbankan keteraturan demi terobosan. Karakteristik Anda didominasi oleh komunikasi ekstrem: Anda sangat komunikatif dan sering sekali berbicara atau menjadi orang yang paling aktif dalam percakapan. Anda sangat mudah bergaul dan tidak pendiam, membuat Anda sangat efektif dalam kerja sama dan penyebaran pengetahuan. Ideologi Anda adalah sangat orisinil dan secara konsisten membawa ide-ide baru, dengan imajinasi yang sangat hidup dan kaya, mendasari fokus R&D yang intensif. Meskipun Neuroticism Anda sangat rendah, yaitu sangat santai dan jarang merasa khawatir atau gugup sehingga menumbuhkan budaya perusahaan yang tangguh; Anda menunjukkan Conscientiousness yang rendah: Anda cenderung kurang teliti dan kadang melewatkan beberapa detail dalam pekerjaan. Anda kurang efektif dan efisien dalam hal keteraturan formal, namun justru fleksibilitas inilah yang memungkinkan Anda merespons perubahan pasar dengan cepat. Dalam interaksi, Anda cenderung kurang perhatian dan kurang mudah memaafkan, mencerminkan orientasi yang lebih kompetitif dan fokus pada hasil inovasi.",
    "Strategic Risk Navigators": "Klaster Strategic Risk Navigators adalah pelaku pasar yang paling proaktif, didorong oleh toleransi risiko yang sangat tinggi dan keinginan untuk menantang struktur konvensional demi mencapai inovasi. Klaster ini dicirikan oleh ide yang sangat tinggi: Anda sangat orisinil dan secara konsisten membawa ide-ide baru, dengan imajinasi yang sangat hidup. Profil ini mendukung keinginan kuat untuk berinovasi dan mengambil proyek berisiko. Secara emosional, Anda sangat santai dan sangat mampu mengatasi stres dengan baik, karena Anda jarang merasa khawatir atau gugup. Toleransi risiko yang tinggi inilah yang membuat Anda responsif terhadap dinamika pasar. Dalam gaya bekerja, Anda cenderung kurang teliti dan kurang efektif dan efisien dalam penyelesaian tugas secara formal. Sifat ini dipandang sebagai keunggulan karena memungkinkan Anda melewati struktur kaku dan fokus pada kecepatan. Anda biasanya komunikatif dan sering berbicara serta mudah bergaul, memfasilitasi orientasi pasar yang proaktif. Dalam hubungan, Anda cenderung kurang perhatian dan kurang mudah memaafkan, menunjukkan kesediaan untuk mempertanyakan metode konvensional demi inovasi strategis.",
    "Multidimensional Service Innovators": "Klaster Multidimensional Service Innovators adalah klaster layanan yang paling beragam dan adaptif. Mereka memiliki dorongan inovasi yang kuat, dan variasi dalam budaya internal memungkinkan berbagai strategi dalam membangun jaringan dan mengelola proses. Karakteristik inovasi Anda sangat orisinil dan secara konsisten membawa ide-ide baru, didukung oleh imajinasi yang sangat hidup, yang mencerminkan afinitas penelitian yang tinggi. Dalam hal Extraversion, klaster ini menunjukkan variasi dari kadang-kadang komunikatif hingga sangat komunikatif dan sangat mudah bergaul. Variasi ini memprioritaskan pentingnya kerja sama dan jaringan untuk pertumbuhan bisnis. Klaster ini menunjukkan fleksibilitas dalam Conscientiousness: Anda bisa kurang teliti untuk kecepatan, tetapi juga bisa biasanya bekerja dengan teliti ketika proses sistematis dibutuhkan. Keragaman terbesar terlihat pada Neuroticism (dari jarang merasa khawatir hingga sering merasa khawatir) dan Agreeableness (dari tidak penuh perhatian hingga kadang-kadang perhatian dan baik hati), menunjukkan variasi dalam budaya perusahaan dan pendekatan dalam mengelola tantangan.",
    "Focused Tech Innovators": "Klaster Focused Tech Innovators ahli dalam menyatukan keunggulan teknologi dengan fleksibilitas proses dan ketahanan mental yang sangat dibutuhkan di pasar yang cepat. Anda dicirikan oleh ide yang sangat tinggi: Anda sangat orisinil dan secara konsisten membawa ide-ide baru dengan imajinasi yang sangat hidup. Hal ini memastikan fondasi teknologi dan kreativitas yang kuat. Klaster Anda juga menunjukkan resistensi stres yang baik karena tidak terlalu sering khawatir dan tidak mudah gugup, serta cukup baik dalam mengatasi stres. Resiliensi ini mendukung efektivitas di bawah tekanan. Meskipun Anda cenderung kurang teliti dan kadang melewatkan beberapa detail dalam pekerjaan, kekurangan Conscientiousness ini justru diubah menjadi keunggulan karena mendorong fleksibilitas dalam desain proyek dan proses, memungkinkan adaptasi yang sangat cepat terhadap perubahan pasar. Anda memiliki keterampilan relasional yang adaptif, di mana tim bisa dari kurang komunikatif hingga biasanya komunikatif, dan dari kadang-kadang perhatian hingga biasanya penuh perhatian. Variasi ini mendukung suasana kerja yang kooperatif dan gesit dalam menanggapi dinamika pasar."
}

In [ ]:
qb_text_mapping = {
    "OPE-1": { # Saya adalah seseorang yang orisinil dan membawa ide-ide baru [cite: 370, 371]
        1: "Saya tidak merasa diri saya orisinil dan jarang membawa ide-ide baru.", # [cite: 371]
        2: "Saya kadang merasa sulit untuk menghasilkan ide baru dan tidak selalu menunjukkan keorisinalan.", # [cite: 372]
        3: "Saya sesekali menunjukkan keorisinalan dan terkadang membawa ide-ide baru.", # [cite: 373]
        4: "Saya sering menunjukkan keorisinalan dan sering membawa ide-ide baru.", # [cite: 373]
        5: "Saya sangat orisinil dan secara konsisten membawa ide-ide baru." # [cite: 374]
    },
    "OPE-2": { # Saya adalah seseorang yang menghargai pengalaman artistik [cite: 375]
        1: "Saya tidak menghargai pengalaman artistik dan jarang merasa tertarik pada hal-hal yang bersifat seni.", # [cite: 375]
        2: "Saya kurang menghargai pengalaman artistik dan tidak sering merasa terlibat dalam kegiatan atau apresiasi seni.", # [cite: 376]
        3: "Saya kadang-kadang menghargai pengalaman artistik, tergantung situasi atau konteksnya.", # [cite: 377]
        4: "Saya sering menghargai pengalaman artistik dan menikmati berbagai bentuk kegiatan atau karya seni.", # [cite: 377]
        5: "Saya sangat menghargai pengalaman artistik dan secara konsisten menikmati serta mencari kesempatan untuk terlibat dalam seni." # [cite: 378]
    },
    "OPE-3": { # Saya adalah seseorang yang memiliki imajinasi yang jelas/hidup (atau kaya) dan penuh fantasi [cite: 379, 380]
        1: "Saya tidak memiliki imajinasi yang hidup dan jarang membayangkan hal-hal secara kreatif atau fantastis.", # [cite: 381]
        2: "Saya memiliki imajinasi yang kurang hidup dan tidak sering berfantasi atau membayangkan hal-hal secara mendetail.", # [cite: 382]
        3: "Saya memiliki imajinasi yang kadang-kadang hidup, dan sesekali membayangkan hal-hal secara kreatif atau fantastis.", # [cite: 383]
        4: "Saya memiliki imajinasi yang cukup hidup dan sering membayangkan hal-hal secara kreatif dan penuh fantasi.", # [cite: 384]
        5: "Saya memiliki imajinasi yang sangat hidup dan kaya, serta secara konsisten membayangkan hal-hal secara kreatif, mendetail, dan penuh fantasi." # [cite: 385]
    },
    "CON-1": { # Saya adalah seseorang yang bekerja secara teliti (atau menyeluruh) [cite: 386]
        1: "Saya tidak teliti dalam bekerja dan sering melewatkan detail penting.", # [cite: 387]
        2: "Saya kurang teliti dan kadang melewatkan beberapa detail dalam pekerjaan.", # [cite: 387]
        3: "Saya kadang-kadang teliti, tetapi tidak selalu konsisten dalam memperhatikan detail.", # [cite: 388]
        4: "Saya biasanya bekerja dengan teliti dan umumnya memperhatikan detail dengan baik.", # [cite: 389]
        5: "Saya sangat teliti dan selalu memperhatikan setiap detail secara menyeluruh dalam pekerjaan." # [cite: 390]
    },
    "CON-2": { # Saya adalah seseorang yang cenderung malas (atau agak malas) [cite: 391]
        1: "Saya tidak merasa malas dan selalu berusaha aktif dalam menyelesaikan pekerjaan.", # [cite: 391]
        2: "Saya jarang merasa malas dan biasanya tetap berinisiatif untuk menyelesaikan tugas.", # [cite: 392]
        3: "Saya kadang-kadang merasa malas, tetapi hal itu tidak terlalu sering mempengaruhi pekerjaan saya.", # [cite: 393]
        4: "Saya sering merasa malas dan hal itu kadang membuat saya menunda pekerjaan.", # [cite: 394]
        5: "Saya sangat cenderung malas dan sering menunda atau menghindari pekerjaan yang seharusnya saya lakukan." # [cite: 395]
    },
    "CON-3": { # Saya adalah seseorang yang menyelesaikan tugas secara efektif dan efisien [cite: 396]
        1: "Saya tidak menyelesaikan tugas secara efektif dan efisien, dan sering membutuhkan waktu lebih lama dari yang diperlukan.", # [cite: 397]
        2: "Saya kurang efektif dan efisien dalam menyelesaikan tugas dan kadang bekerja lebih lambat dari yang seharusnya.", # [cite: 398]
        3: "Saya kadang efektif dan efisien, tetapi tidak selalu konsisten dalam menyelesaikan tugas.", # [cite: 399]
        4: "Saya biasanya menyelesaikan tugas dengan efektif dan efisien dan mampu mengatur waktu dengan baik.", # [cite: 400]
        5: "Saya sangat efektif dan efisien dalam menyelesaikan tugas dan secara konsisten bekerja dengan cepat serta terorganisir." # [cite: 401]
    },
    "EXT-1": { # Saya adalah seseorang yang komunikatif dan banyak bicara (atau cerewet) [cite: 402]
        1: "Saya tidak komunikatif dan jarang berbicara dalam berbagai situasi.", # [cite: 403]
        2: "Saya kurang komunikatif dan tidak terlalu sering berbicara atau terlibat dalam percakapan.", # [cite: 403]
        3: "Saya kadang-kadang komunikatif, tetapi frekuensi berbicara saya bervariasi tergantung situasinya.", # [cite: 404]
        4: "Saya biasanya komunikatif dan sering berbicara atau terlibat dalam percakapan.", # [cite: 404]
        5: "Saya sangat komunikatif dan sering sekali berbicara atau menjadi orang yang paling aktif dalam percakapan." # [cite: 405]
    },
    "EXT-2": { # Saya adalah seseorang yang bisa keluar sendiri dan bersosialisasi (atau mudah bergaul) [cite: 406, 407]
        1: "Saya tidak mudah bergaul dan jarang ingin keluar sendiri atau bersosialisasi.", # [cite: 408]
        2: "Saya kurang suka bersosialisasi dan tidak terlalu nyaman keluar atau bergaul dengan orang baru.", # [cite: 409]
        3: "Saya kadang-kadang mau bersosialisasi atau keluar sendiri, tetapi itu tergantung situasi.", # [cite: 410]
        4: "Saya mudah bergaul dan sering merasa nyaman untuk keluar sendiri serta bersosialisasi.", # [cite: 411]
        5: "Saya sangat mudah bergaul dan sangat nyaman keluar sendiri serta aktif bersosialisasi dengan banyak orang." # [cite: 412]
    },
    "EXT-3": { # Saya adalah seseorang yang pendiam (atau tertutup) [cite: 413]
        1: "Saya tidak pendiam dan sering berbicara atau terbuka dalam berbagai situasi.", # [cite: 414]
        2: "Saya kurang pendiam dan biasanya cukup terbuka untuk berbicara dengan orang lain.", # [cite: 415]
        3: "Saya kadang pendiam, tetapi di beberapa situasi saya bisa cukup terbuka.", # [cite: 416]
        4: "Saya biasanya pendiam dan lebih sering memilih untuk tidak banyak berbicara.", # [cite: 417]
        5: "Saya sangat pendiam dan lebih suka tertutup, jarang berbicara atau mengungkapkan hal kepada orang lain." # [cite: 418]
    },
    "AGR-1": { # Saya adalah seseorang yang kadang-kadang agak kasar (atau sedikit tidak sopan) kepada orang lain [cite: 419, 420]
        1: "Saya tidak pernah bersikap kasar atau tidak sopan kepada orang lain.", # [cite: 421]
        2: "Saya jarang bersikap kasar atau tidak sopan, dan umumnya tetap menjaga sikap.", # [cite: 422]
        3: "Saya kadang-kadang bisa bersikap agak kasar, tetapi tidak terlalu sering.", # [cite: 423]
        4: "Saya cukup sering bersikap agak kasar atau sedikit tidak sopan dalam beberapa situasi.", # [cite: 424]
        5: "Saya sering bersikap agak kasar atau tidak sopan kepada orang lain dalam berbagai situasi." # [cite: 425]
    },
    "AGR-2": { # Saya adalah seseorang yang mudah memaafkan (atau bisa memaafkan) [cite: 426]
        1: "Saya sulit memaafkan dan jarang melupakan kesalahan orang lain.", # [cite: 426]
        2: "Saya kurang mudah memaafkan, meskipun kadang bisa melakukannya setelah waktu yang cukup lama.", # [cite: 427]
        3: "Saya kadang-kadang bisa memaafkan, tergantung situasi dan tingkat kesalahannya.", # [cite: 428]
        4: "Saya biasanya mudah memaafkan dan tidak menyimpan rasa kesal terlalu lama.", # [cite: 428]
        5: "Saya sangat mudah memaafkan dan cepat mengesampingkan kesalahan orang lain tanpa menyimpannya di hati." # [cite: 429]
    },
    "AGR-3": { # Saya adalah seseorang yang penuh perhatian dan baik hati terhadap orang lain [cite: 430]
        1: "Saya tidak penuh perhatian dan jarang menunjukkan kebaikan hati kepada orang lain.", # [cite: 431]
        2: "Saya kurang perhatian dan tidak selalu menunjukkan kebaikan hati dalam interaksi saya.", # [cite: 432]
        3: "Saya kadang-kadang perhatian dan baik hati, tetapi tidak selalu konsisten.", # [cite: 432]
        4: "Saya biasanya penuh perhatian dan sering menunjukkan kebaikan hati kepada orang lain.", # [cite: 433]
        5: "Saya sangat penuh perhatian dan secara konsisten menunjukkan kebaikan hati kepada orang lain." # [cite: 434]
    },
    "NEU-1": { # Saya adalah seseorang yang sering khawatir [cite: 435]
        1: "Saya jarang merasa khawatir dan umumnya tenang dalam berbagai situasi.", # [cite: 435]
        2: "Saya tidak terlalu sering khawatir, meskipun sesekali bisa merasa cemas.", # [cite: 436]
        3: "Saya kadang-kadang merasa khawatir, tergantung situasi atau tekanan yang dihadapi.", # [cite: 436]
        4: "Saya sering merasa khawatir dalam berbagai situasi, terutama ketika menghadapi ketidakpastian.", # [cite: 437]
        5: "Saya sangat sering merasa khawatir dan mudah cemas dalam banyak keadaan." # [cite: 438]
    },
    "NEU-2": { # Saya adalah seseorang yang mudah gugup [cite: 439]
        1: "Saya jarang merasa gugup dan biasanya tetap tenang dalam berbagai situasi.", # [cite: 439]
        2: "Saya tidak mudah gugup, meskipun sesekali bisa merasa tegang dalam kondisi tertentu.", # [cite: 440]
        3: "Saya kadang-kadang gugup, tergantung konteks dan situasinya.", # [cite: 440]
        4: "Saya cukup mudah gugup dan sering merasa tegang dalam beberapa situasi.", # [cite: 441]
        5: "Saya sangat mudah gugup dan sering merasa tegang bahkan dalam situasi yang ringan sekalipun." # [cite: 442]
    },
    "NEU-3": { # Saya adalah seseorang yang santai dan dapat mengatasi stres dengan baik [cite: 443, 444]
        1: "Saya tidak santai dan sering kesulitan mengatasi stres.", # [cite: 444]
        2: "Saya kurang santai dan kadang merasa sulit mengatasi stres.", # [cite: 445]
        3: "Saya kadang santai, tetapi kemampuan saya dalam mengatasi stres bervariasi tergantung situasinya.", # [cite: 445]
        4: "Saya biasanya santai dan cukup baik dalam mengatasi stres.", # [cite: 446]
        5: "Saya sangat santai dan sangat mampu mengatasi stres dengan baik dalam berbagai situasi." # [cite: 446]
    }
}

In [ ]:
def build_user_qb_document(row, question_map):
    texts = []

    for col, mapping in question_map.items():
        val = row.get(col)
        if pd.notna(val) and int(val) in mapping:
            texts.append(mapping[int(val)])

    return " ".join(texts)

In [ ]:
df["qb_user_text"] = df.apply(
    lambda row: build_user_qb_document(row, qb_text_mapping),
    axis=1
)

In [ ]:
df["qb_user_text"]

In [ ]:
# corpus = [cluster1, cluster2, ..., user_text]
cluster_names = list(cluster_descriptions.keys())
cluster_docs  = list(cluster_descriptions.values())

In [ ]:
def tfidf_top3_clusters_qB(user_text, cluster_docs, cluster_names):

    if not isinstance(user_text, str) or user_text.strip() == "":
        return [], []

    corpus = cluster_docs + [user_text]

    vectorizer = TfidfVectorizer(
        stop_words="english",
        ngram_range=(1,2)
    )

    tfidf_matrix = vectorizer.fit_transform(corpus)

    # similarity user vs cluster
    similarity = cosine_similarity(
        tfidf_matrix[-1],
        tfidf_matrix[:-1]
    )[0]

    # ranking
    ranked_idx = similarity.argsort()[::-1]

    top3_idx = ranked_idx[:3]

    top3_clusters = [cluster_names[i] for i in top3_idx]
    top3_scores   = [(cluster_names[i], similarity[i]) for i in top3_idx]

    return top3_clusters, top3_scores

In [ ]:
tfidf_results = df["qb_user_text"].apply(
    lambda x: tfidf_top3_clusters_qB(
        x,
        cluster_docs,
        cluster_names
    )
)

df["top3_cluster_tfidf_qB"] = tfidf_results.apply(lambda x: x[0])
df["top3_cluster_tfidf_qB_score"] = tfidf_results.apply(lambda x: x[1])

In [ ]:
def pick_best_cluster_from_tfidf(row):
    scores = row["top3_cluster_tfidf_qB_score"]

    if not scores:
        return np.nan, np.nan

    best_cluster, best_score = max(scores, key=lambda x: x[1])

    return best_cluster, best_score


best_qB = df.apply(pick_best_cluster_from_tfidf, axis=1)

df["cluster_top3_best_qB"] = best_qB.apply(lambda x: x[0])
df["cluster_top3_best_qB_score"] = best_qB.apply(lambda x: x[1])

In [ ]:
df[[
    "user_id",
    "top3_clusters_qB",                 # hasil rule
    "top3_cluster_tfidf_qB",             # hasil TF-IDF
    "top3_cluster_tfidf_qB_score",
    "cluster_top3_best_qB",
    "cluster_top3_best_qB_score"
]]

## CLUSTER -> BEST SECTOR

berarti ini nanti bukan top5_clusters_qB lagi yang dimatch sama sektor tapi yang top 3, setelah itu tfidf akan menghasilkan top 1 clusternya disimpan ke field cluster_top5_best_qB

In [ ]:
def top3_clusters_with_matched_sectors_qB(row):

    top3_clusters_qB = row["top3_cluster_tfidf_qB"]          # list cluster nama
    top3_sectors_qA  = row["top3_tfidf_sector"]            # list sektor Top-3 tfi-df qA

    # validasi
    if not isinstance(top3_clusters_qB, list) or not isinstance(top3_sectors_qA, list):
        return {}

    hasil = {}

    for cluster_name in top3_clusters_qB:

        # pastikan cluster ada di mapping
        if cluster_name in cluster_mapping:

            # ambil semua sektor cluster tsb
            sektor_cluster = cluster_mapping[cluster_name]

            # cocokkan dgn top3 euclid qA
            sektor_match = [s for s in sektor_cluster if s in top3_sectors_qA]

            # jika ada kecocokan → simpan
            if len(sektor_match) > 0:
                hasil[cluster_name] = sektor_match

    return hasil

# simpan hasil ke dataframe
df["top3_cluster_to_sector_qB"] = df.apply(top3_clusters_with_matched_sectors_qB, axis=1)

## OUTPUT UNTUK MEMBANDINGKAN A VS B

In [ ]:
df[["user_id",
    "top3_euclid_qA",
    "top3_tfidf_sector",
    "cluster_top5_best_qA",
    "cluster_top5_ranking_qA",
    "top5_cluster_to_sector_qA",
    "cluster_top3_best_qB",
    "top3_cluster_tfidf_qB_score",
    "top3_cluster_to_sector_qB"
    ]]

In [ ]:
df

# COMPARISSON RECOMMENDATION BAGIAN A VS B

In [ ]:
def get_recommended_cluster_refined(row):

    qA = row.get("cluster_top5_best_qA", [])
    qB = row.get("cluster_top3_best_qB", [])
    mapping = row.get("top5_cluster_to_sector_qA", {}) or {}

    # Normalisasi input
    if isinstance(qA, str):
        qA = [x.strip() for x in qA.split(",")] if qA.strip() else []
    if isinstance(qB, str):
        qB = [x.strip() for x in qB.split(",")] if qB.strip() else []

    # Jika salah satu kosong
    if not qA or not qB:
        return ["Belum dapat menentukan cluster yang sesuai."]

    # RULE 1 — Irisan
    intersection = [c for c in qA if c in qB]
    if intersection:
        initial = intersection[:]
    else:
        # RULE 2 — Ambil terbaik masing-masing
        initial = [qA[0], qB[0]]

    # Validasi kandidat awal
    valid_initial = [
        c for c in initial
        if c in mapping and len(mapping.get(c, [])) > 0
    ]

    final_candidates = valid_initial.copy()

    # Semua kandidat yang tersedia
    all_clusters = list(dict.fromkeys(qA + qB))

    def sektor_baru(cluster, sektor_terwakili):
        return set(mapping.get(cluster, [])) - sektor_terwakili

    # Substitusi jika ada kandidat invalid
    for invalid in set(initial) - set(valid_initial):

        sektor_terwakili = set()
        for c in final_candidates:
            sektor_terwakili.update(mapping.get(c, []))

        best_choice = None
        best_gain = -1

        for candidate in all_clusters:
            if candidate in final_candidates:
                continue
            if candidate not in mapping:
                continue

            gain = len(sektor_baru(candidate, sektor_terwakili))
            if gain > best_gain:
                best_gain = gain
                best_choice = candidate

        if best_choice:
            final_candidates.append(best_choice)

    # RULE tambahan — cover sektor user
    sektor_user = set(row.get("top3_tfidf_sector", []))

    sektor_terwakili = set()
    for c in final_candidates:
        sektor_terwakili.update(mapping.get(c, []))

    sektor_kurang = sektor_user - sektor_terwakili

    if sektor_kurang:
        best_choice = None
        best_gain = -1

        for candidate in all_clusters:
            if candidate in final_candidates:
                continue
            if candidate not in mapping:
                continue

            gain = len(set(mapping[candidate]) & sektor_kurang)
            if gain > best_gain:
                best_gain = gain
                best_choice = candidate

        if best_choice:
            final_candidates.append(best_choice)

    return final_candidates


In [ ]:
df["recommended_cluster"] = df.apply(
    get_recommended_cluster_refined,
    axis=1
)

In [ ]:
df[["user_id", "recommended_cluster", "top3_tfidf_sector"]]

## CODE TRACKING

In [ ]:
# def get_recommended_cluster_refined_v4(row):
#     log = []

#     qA = row.get("cluster_top5_best_qA", [])
#     qB = row.get("cluster_top5_best_qB", [])
#     mapping = row.get("top5_cluster_to_sector_qA", {}) or {}
#     rankingA = row.get("cluster_top5_ranking_qA", [])

#     # Normalisasi
#     if isinstance(qA, str):
#         qA = [x.strip() for x in qA.split(",")] if qA.strip() else []
#     if isinstance(qB, str):
#         qB = [x.strip() for x in qB.split(",")] if qB.strip() else []

#     log.append(f"Q-A list: {qA}")
#     log.append(f"Q-B list: {qB}")

#     if not qA or not qB:
#         result = ["Belum dapat menentukan cluster yang sesuai."]
#         log.append("Rule 3 karena salah satu kosong.")
#         return result, "\n".join(log)

#     # RULE 1 — irisannya
#     intersection = [c for c in qA if c in qB]
#     if intersection:
#         log.append("Rule 1 — Ada irisan.")
#         initial = intersection[:]
#     else:
#         log.append("Rule 2 — Tidak ada irisan.")
#         initial = [qA[0], qB[0]]
#         log.append(f"Kandidat awal Rule 2: {initial}")

#     log.append("\n--- Tahap Validasi Kandidat Initial ---")

#     valid_initial = []
#     invalid_initial = []

#     for c in initial:
#         if c in mapping and len(mapping.get(c, [])) > 0:
#             valid_initial.append(c)
#         else:
#             invalid_initial.append(c)

#     log.append(f"Valid initial clusters: {valid_initial}")
#     log.append(f"Invalid initial clusters: {invalid_initial}")

#     final_candidates = valid_initial.copy()

#     # Kandidat yang tersedia untuk substitusi
#     all_clusters = list(dict.fromkeys(qA + qB))

#     def sektor_baru_dari(cluster, sektor_terwakili):
#         return set(mapping.get(cluster, [])) - sektor_terwakili

#     # Substitusi hanya untuk invalid
#     for invalid in invalid_initial:
#         log.append(f"\nSubstitusi untuk cluster invalid: {invalid}")

#         sektor_terwakili = set()
#         for c in final_candidates:
#             sektor_terwakili.update(mapping.get(c, []))

#         best_choice = None
#         best_new_sector = -1

#         for candidate in all_clusters:
#             if candidate == invalid:
#                 continue
#             if candidate in final_candidates:
#                 continue
#             if candidate not in mapping:
#                 continue

#             new_sector_count = len(sektor_baru_dari(candidate, sektor_terwakili))

#             if new_sector_count > best_new_sector:
#                 best_new_sector = new_sector_count
#                 best_choice = candidate

#         if best_choice:
#             log.append(f" → Diganti dengan: {best_choice} (menambah {best_new_sector} sektor baru)")
#             final_candidates.append(best_choice)
#         else:
#             log.append(" → Tidak ada substitusi yang valid.")

#     # --------------------------------------------------------------------
#     # ⭐ NEW RULE: Tambah cluster jika masih ada sektor user yang belum tercover
#     # --------------------------------------------------------------------
#     sektor_user = set(row.get("top3_euclid_qA", []))

#     sektor_terwakili = set()
#     for c in final_candidates:
#         sektor_terwakili.update(mapping.get(c, []))

#     sektor_kurang = sektor_user - sektor_terwakili

#     if sektor_kurang:
#         log.append(f"Ada sektor yang belum terwakili: {sektor_kurang}")

#         best_choice = None
#         best_gain = -1

#         for candidate in all_clusters:
#             if candidate in final_candidates:
#                 continue
#             if candidate not in mapping:
#                 continue

#             sektor_candidate = set(mapping[candidate])
#             gain = len(sektor_candidate & sektor_kurang)

#             if gain > best_gain:
#                 best_gain = gain
#                 best_choice = candidate

#         if best_choice:
#             log.append(f"Menambahkan cluster kedua karena sektor kurang → {best_choice} (gain={best_gain})")
#             final_candidates.append(best_choice)
#         else:
#             log.append("Tidak ada cluster tambahan yang bisa menutupi sektor yang kurang.")

#     log.append(f"\nFinal Result: {final_candidates}")
#     return final_candidates, "\n".join(log)

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[0])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][0])

# print("\n")
# print(df['cluster_top5_ranking_qA'][0])

# print("\n")
# df['top5_cluster_to_sector_qA'][0]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[1])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][1])

# print("\n")
# print(df['cluster_top5_ranking_qA'][1])

# print("\n")
# df['top5_cluster_to_sector_qA'][1]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[2])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][2])

# print("\n")
# print(df['cluster_top5_ranking_qA'][2])

# print("\n")
# df['top5_cluster_to_sector_qA'][2]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[3])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][3])

# print("\n")
# print(df['cluster_top5_ranking_qA'][3])

# print("\n")
# df['top5_cluster_to_sector_qA'][3]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[4])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][4])

# print("\n")
# print(df['cluster_top5_ranking_qA'][4])

# print("\n")
# df['top5_cluster_to_sector_qA'][4]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[5])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][5])

# print("\n")
# print(df['cluster_top5_ranking_qA'][5])

# print("\n")
# df['top5_cluster_to_sector_qA'][5]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[6])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][6])

# print("\n")
# print(df['cluster_top5_ranking_qA'][6])

# print("\n")
# df['top5_cluster_to_sector_qA'][6]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[7])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][7])

# print("\n")
# print(df['cluster_top5_ranking_qA'][7])

# print("\n")
# df['top5_cluster_to_sector_qA'][7]

In [ ]:
# recommended, log = get_recommended_cluster_refined_v4(df.iloc[8])

# print("RECOMMENDED:", recommended)
# print("\nLOG:", log)

# print("\n")
# print(df['top3_euclid_qA'][8])

# print("\n")
# print(df['cluster_top5_ranking_qA'][8])

# print("\n")
# df['top5_cluster_to_sector_qA'][8]